<a href="https://colab.research.google.com/github/ShaliniAnandaPhD/Neuron/blob/main/Tutorial_14_Custom_Agent_Type.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


Tutorial 14 Part 1: Custom Agent Types
====================================================

Building the foundation for creating specialized, domain-specific agents
with custom capabilities, behaviors, and expert knowledge systems.

What you'll learn:

• Agent specialization patterns and architectures

• Domain-specific capability frameworks

• Expert knowledge integration systems

• Custom behavior definition and execution

• Agent trait and skill composition


In [1]:
import uuid
import time
import json
import threading
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Tuple, Callable, Union, Type
from enum import Enum
from collections import defaultdict, deque
from abc import ABC, abstractmethod
import inspect
import functools

In [2]:
print("Tutorial 14 Part 1: Custom Agent Foundation Framework")
print("=" * 55)
print("Building specialized domain-specific agent capabilities...")
print()

Tutorial 14 Part 1: Custom Agent Foundation Framework
Building specialized domain-specific agent capabilities...



In [13]:
# CORE AGENT SPECIALIZATION TYPES AND CONCEPTS
# =============================================================================

class AgentDomain(Enum):
    """Primary domains for agent specialization"""
    FINANCIAL = "financial"               # Financial analysis, trading, accounting
    HEALTHCARE = "healthcare"             # Medical analysis, diagnosis, treatment
    LEGAL = "legal"                       # Legal research, contract analysis
    EDUCATION = "education"               # Teaching, curriculum, assessment
    ENGINEERING = "engineering"           # Technical design, analysis, optimization
    CREATIVE = "creative"                 # Content creation, design, art
    RESEARCH = "research"                 # Data analysis, scientific research
    CUSTOMER_SERVICE = "customer_service" # Support, communication, problem solving
    SALES = "sales"                       # Lead generation, negotiation, closing
    OPERATIONS = "operations"             # Process management, logistics, efficiency

class CapabilityType(Enum):
    """Types of capabilities agents can possess"""
    ANALYTICAL = "analytical"             # Data analysis, pattern recognition
    COMMUNICATIVE = "communicative"       # Language, negotiation, presentation
    CREATIVE = "creative"                 # Content generation, design, innovation
    TECHNICAL = "technical"               # Programming, system administration
    ADVISORY = "advisory"                 # Consultation, recommendation, guidance
    TRANSACTIONAL = "transactional"      # Processing, execution, automation
    MONITORING = "monitoring"             # Observation, alerting, tracking
    LEARNING = "learning"                 # Adaptation, improvement, training

class ExpertiseLevel(Enum):
    """Levels of expertise for different capabilities"""
    NOVICE = 1                           # Basic understanding
    INTERMEDIATE = 2                      # Working knowledge
    ADVANCED = 3                         # Expert knowledge
    MASTER = 4                           # Deep specialization
    AUTHORITY = 5                        # Industry leadership

@dataclass
class AgentCapability:
    """
    Represents a specific capability an agent can possess

    This defines what the agent can do, how well it can do it,
    and the conditions under which it operates best.
    """

    capability_id: str
    name: str
    capability_type: CapabilityType
    expertise_level: ExpertiseLevel

    # Capability definition
    description: str = ""
    prerequisites: List[str] = field(default_factory=list)
    outputs: List[str] = field(default_factory=list)

    # Performance characteristics
    accuracy_rating: float = 0.8         # How accurate this capability is (0-1)
    speed_rating: float = 0.8            # How fast this capability executes (0-1)
    reliability_rating: float = 0.8      # How consistently this capability works (0-1)

    # Operational parameters
    max_concurrent_uses: int = 1          # How many simultaneous uses
    cooldown_seconds: float = 0.0        # Time between uses
    resource_cost: float = 1.0           # Computational cost (arbitrary units)

    # Usage tracking
    times_used: int = 0
    success_count: int = 0
    last_used: float = 0.0

    # Implementation
    execution_function: Optional[Callable] = None
    validation_function: Optional[Callable] = None

    def can_execute(self) -> bool:
        """Check if capability can be executed now"""
        current_time = time.time()

        # Check cooldown
        if current_time - self.last_used < self.cooldown_seconds:
            return False

        # Check if execution function exists
        if not self.execution_function:
            return False

        return True

    def execute(self, *args, **kwargs) -> Dict[str, Any]:
        """Execute the capability"""
        if not self.can_execute():
            return {
                'success': False,
                'error': 'Capability not available for execution',
                'capability_id': self.capability_id
            }

        start_time = time.time()

        try:
            # Execute the capability
            result = self.execution_function(*args, **kwargs)

            # Update usage statistics
            self.times_used += 1
            self.success_count += 1
            self.last_used = time.time()

            # Wrap result
            execution_time = time.time() - start_time

            return {
                'success': True,
                'result': result,
                'capability_id': self.capability_id,
                'execution_time': execution_time,
                'expertise_level': self.expertise_level.value
            }

        except Exception as e:
            self.times_used += 1
            self.last_used = time.time()

            return {
                'success': False,
                'error': str(e),
                'capability_id': self.capability_id,
                'execution_time': time.time() - start_time
            }

    def get_success_rate(self) -> float:
        """Calculate success rate for this capability"""
        if self.times_used == 0:
            return 0.0
        return self.success_count / self.times_used

    def get_performance_metrics(self) -> Dict[str, Any]:
        """Get comprehensive performance metrics"""
        return {
            'capability_id': self.capability_id,
            'name': self.name,
            'expertise_level': self.expertise_level.value,
            'times_used': self.times_used,
            'success_rate': self.get_success_rate(),
            'accuracy_rating': self.accuracy_rating,
            'speed_rating': self.speed_rating,
            'reliability_rating': self.reliability_rating,
            'last_used': self.last_used,
            'average_cost': self.resource_cost
        }

@dataclass
class AgentTrait:
    """
    Represents a behavioral trait that influences how an agent operates

    Traits affect the agent's decision-making, communication style,
    and approach to problem-solving.
    """

    trait_id: str
    name: str
    description: str

    # Trait intensity (0.0 to 1.0)
    intensity: float = 0.5

    # Behavioral modifiers
    decision_modifier: float = 0.0        # How this affects decision making (-1 to 1)
    communication_modifier: float = 0.0   # How this affects communication (-1 to 1)
    risk_modifier: float = 0.0           # How this affects risk tolerance (-1 to 1)
    speed_modifier: float = 0.0          # How this affects operation speed (-1 to 1)

    # Trait activation conditions
    activation_contexts: List[str] = field(default_factory=list)
    inhibition_contexts: List[str] = field(default_factory=list)

    def is_active(self, context: Dict[str, Any]) -> bool:
        """Check if trait should be active in given context"""

        # Check inhibition contexts first
        for inhibitor in self.inhibition_contexts:
            if inhibitor in context:
                return False

        # Check activation contexts
        if not self.activation_contexts:
            return True  # Active by default if no specific contexts

        for activator in self.activation_contexts:
            if activator in context:
                return True

        return False

    def apply_modifiers(self, base_value: float, modifier_type: str) -> float:
        """Apply trait modifiers to a base value"""

        modifier_map = {
            'decision': self.decision_modifier,
            'communication': self.communication_modifier,
            'risk': self.risk_modifier,
            'speed': self.speed_modifier
        }

        modifier = modifier_map.get(modifier_type, 0.0)

        # Apply modifier with intensity scaling
        adjusted_modifier = modifier * self.intensity

        # Apply modifier to base value
        modified_value = base_value + adjusted_modifier

        # Keep within reasonable bounds
        return max(0.0, min(1.0, modified_value))

class DomainKnowledge:
    """
    Container for domain-specific knowledge and expertise

    This represents the specialized knowledge that makes an agent
    an expert in a particular domain.
    """

    def __init__(self, domain: AgentDomain):
        self.domain = domain
        self.knowledge_base = {}
        self.facts = []
        self.rules = []
        self.procedures = []
        self.terminology = {}
        self.best_practices = []

        print(f"📚 Domain knowledge initialized for: {domain.value}")

    def add_fact(self, fact_id: str, description: str, confidence: float = 1.0,
                source: str = "", tags: List[str] = None):
        """Add a factual piece of knowledge"""

        fact = {
            'fact_id': fact_id,
            'description': description,
            'confidence': confidence,
            'source': source,
            'tags': tags or [],
            'added_at': time.time()
        }

        self.facts.append(fact)
        self.knowledge_base[fact_id] = fact

        print(f"   📝 Added fact: {description[:50]}...")

    def add_rule(self, rule_id: str, condition: str, action: str,
                confidence: float = 1.0, priority: int = 50):
        """Add a rule to the knowledge base"""

        rule = {
            'rule_id': rule_id,
            'condition': condition,
            'action': action,
            'confidence': confidence,
            'priority': priority,
            'added_at': time.time(),
            'times_applied': 0
        }

        self.rules.append(rule)
        self.knowledge_base[rule_id] = rule

        print(f"   📋 Added rule: {condition} → {action}")

    def add_procedure(self, procedure_id: str, name: str, steps: List[str],
                     prerequisites: List[str] = None, expected_outcome: str = ""):
        """Add a procedure to the knowledge base"""

        procedure = {
            'procedure_id': procedure_id,
            'name': name,
            'steps': steps,
            'prerequisites': prerequisites or [],
            'expected_outcome': expected_outcome,
            'added_at': time.time(),
            'times_executed': 0
        }

        self.procedures.append(procedure)
        self.knowledge_base[procedure_id] = procedure

        print(f"   📋 Added procedure: {name} ({len(steps)} steps)")

    def add_terminology(self, term: str, definition: str, context: str = ""):
        """Add domain-specific terminology"""

        self.terminology[term] = {
            'definition': definition,
            'context': context,
            'added_at': time.time()
        }

        print(f"   📖 Added term: {term}")

    def query_knowledge(self, query: str, knowledge_type: str = "all") -> List[Dict[str, Any]]:
        """Query the knowledge base"""

        results = []
        query_lower = query.lower()

        # Search facts
        if knowledge_type in ["all", "facts"]:
            for fact in self.facts:
                if query_lower in fact['description'].lower():
                    results.append({
                        'type': 'fact',
                        'content': fact,
                        'relevance': self._calculate_relevance(query, fact['description'])
                    })

        # Search rules
        if knowledge_type in ["all", "rules"]:
            for rule in self.rules:
                if (query_lower in rule['condition'].lower() or
                    query_lower in rule['action'].lower()):
                    results.append({
                        'type': 'rule',
                        'content': rule,
                        'relevance': self._calculate_relevance(query, rule['condition'] + " " + rule['action'])
                    })

        # Search procedures
        if knowledge_type in ["all", "procedures"]:
            for procedure in self.procedures:
                if (query_lower in procedure['name'].lower() or
                    any(query_lower in step.lower() for step in procedure['steps'])):
                    results.append({
                        'type': 'procedure',
                        'content': procedure,
                        'relevance': self._calculate_relevance(query, procedure['name'])
                    })

        # Search terminology
        if knowledge_type in ["all", "terminology"]:
            for term, definition in self.terminology.items():
                if (query_lower in term.lower() or
                    query_lower in definition['definition'].lower()):
                    results.append({
                        'type': 'terminology',
                        'content': {'term': term, **definition},
                        'relevance': self._calculate_relevance(query, term + " " + definition['definition'])
                    })

        # Sort by relevance
        results.sort(key=lambda x: x['relevance'], reverse=True)

        return results

    def _calculate_relevance(self, query: str, text: str) -> float:
        """Calculate relevance score between query and text"""
        query_words = query.lower().split()
        text_words = text.lower().split()

        if not query_words:
            return 0.0

        matches = sum(1 for word in query_words if word in text_words)
        return matches / len(query_words)

    def get_statistics(self) -> Dict[str, Any]:
        """Get knowledge base statistics"""
        return {
            'domain': self.domain.value,
            'total_knowledge_items': len(self.knowledge_base),
            'facts_count': len(self.facts),
            'rules_count': len(self.rules),
            'procedures_count': len(self.procedures),
            'terminology_count': len(self.terminology),
            'coverage_areas': list(set(
                tag for fact in self.facts
                for tag in fact.get('tags', [])
            ))
        }

class CustomAgentBase(ABC):
    """
    Abstract base class for custom specialized agents

    This provides the foundation that all custom agents build upon,
    including capability management, trait application, and
    domain knowledge integration.
    """

    def __init__(self, agent_id: str = None, name: str = "",
                 domain: AgentDomain = AgentDomain.RESEARCH):
        self.agent_id = agent_id or str(uuid.uuid4())
        self.name = name or f"CustomAgent_{self.agent_id[:8]}"
        self.domain = domain

        # Core components
        self.capabilities = {}  # capability_id -> AgentCapability
        self.traits = {}       # trait_id -> AgentTrait
        self.domain_knowledge = DomainKnowledge(domain)

        # State and context
        self.current_context = {}
        self.active_traits = {}
        self.execution_history = []

        # Performance tracking
        self.total_operations = 0
        self.successful_operations = 0
        self.expertise_score = 0.0

        print(f"🤖 Custom agent created: {self.name}")
        print(f"   Domain: {domain.value}")
        print(f"   Agent ID: {self.agent_id[:8]}...")

    def add_capability(self, capability: AgentCapability):
        """Add a capability to this agent"""
        self.capabilities[capability.capability_id] = capability
        print(f"   ➕ Added capability: {capability.name} ({capability.expertise_level.name})")

    def add_trait(self, trait: AgentTrait):
        """Add a behavioral trait to this agent"""
        self.traits[trait.trait_id] = trait
        print(f"   🎭 Added trait: {trait.name} (intensity: {trait.intensity:.2f})")

    def update_context(self, context: Dict[str, Any]):
        """Update the agent's operational context"""
        self.current_context.update(context)

        # Update active traits based on new context
        self.active_traits = {
            trait_id: trait
            for trait_id, trait in self.traits.items()
            if trait.is_active(self.current_context)
        }

        print(f"   🔄 Context updated: {len(self.active_traits)} traits now active")

    def execute_capability(self, capability_id: str, *args, **kwargs) -> Dict[str, Any]:
        """Execute a specific capability"""

        if capability_id not in self.capabilities:
            return {
                'success': False,
                'error': f'Capability {capability_id} not found',
                'agent_id': self.agent_id
            }

        capability = self.capabilities[capability_id]

        # Apply trait modifiers to execution
        modified_args, modified_kwargs = self._apply_trait_modifiers(
            capability, args, kwargs
        )

        # Execute capability
        result = capability.execute(*modified_args, **modified_kwargs)

        # Update performance tracking
        self.total_operations += 1
        if result.get('success', False):
            self.successful_operations += 1

        # Record in execution history
        execution_record = {
            'timestamp': time.time(),
            'capability_id': capability_id,
            'result': result,
            'context': self.current_context.copy(),
            'active_traits': list(self.active_traits.keys())
        }
        self.execution_history.append(execution_record)

        # Keep history manageable
        if len(self.execution_history) > 100:
            self.execution_history = self.execution_history[-100:]

        return result

    def _apply_trait_modifiers(self, capability: AgentCapability,
                              args: Tuple, kwargs: Dict[str, Any]) -> Tuple[Tuple, Dict[str, Any]]:
        """Apply active trait modifiers to capability execution"""

        # For now, return unmodified - subclasses can override for specific behavior
        # This is where traits would influence how capabilities are executed

        return args, kwargs

    def get_available_capabilities(self) -> List[str]:
        """Get list of capabilities that can currently be executed"""
        return [
            cap_id for cap_id, capability in self.capabilities.items()
            if capability.can_execute()
        ]

    def get_expertise_assessment(self) -> Dict[str, Any]:
        """Get assessment of agent's expertise and performance"""

        # Calculate overall expertise score
        if self.capabilities:
            avg_expertise = sum(
                cap.expertise_level.value for cap in self.capabilities.values()
            ) / len(self.capabilities)
        else:
            avg_expertise = 0.0

        # Calculate success rate
        success_rate = (self.successful_operations / max(self.total_operations, 1))

        # Combine for overall score
        self.expertise_score = (avg_expertise / 5.0) * 0.5 + success_rate * 0.5

        return {
            'agent_id': self.agent_id,
            'name': self.name,
            'domain': self.domain.value,
            'expertise_score': self.expertise_score,
            'success_rate': success_rate,
            'total_operations': self.total_operations,
            'capabilities_count': len(self.capabilities),
            'traits_count': len(self.traits),
            'active_traits_count': len(self.active_traits),
            'knowledge_base_size': len(self.domain_knowledge.knowledge_base),
            'available_capabilities': self.get_available_capabilities()
        }

    @abstractmethod
    def specialized_operation(self, operation_type: str, **kwargs) -> Dict[str, Any]:
        """
        Abstract method for domain-specific operations

        Each custom agent type must implement this to define
        their specialized behavior and operations.
        """
        pass

# =============================================================================
# TEST THE FOUNDATION CLASSES
# =============================================================================

print("🧪 Testing Custom Agent Foundation...")
print()

# Test AgentCapability
print("📋 Testing AgentCapability:")

def sample_analysis_function(data: str, depth: str = "basic"):
    """Sample analysis capability function"""
    result = {
        'analysis_type': depth,
        'data_processed': len(data),
        'insights': f"Analyzed {len(data)} characters with {depth} depth",
        'confidence': 0.85
    }
    time.sleep(0.1)  # Simulate processing time
    return result

analysis_capability = AgentCapability(
    capability_id="financial_analysis",
    name="Financial Data Analysis",
    capability_type=CapabilityType.ANALYTICAL,
    expertise_level=ExpertiseLevel.ADVANCED,
    description="Analyze financial data and generate insights",
    accuracy_rating=0.92,
    speed_rating=0.85,
    reliability_rating=0.95,
    execution_function=sample_analysis_function
)

print(f"   Created capability: {analysis_capability.name}")
print(f"   Expertise level: {analysis_capability.expertise_level.name}")
print(f"   Can execute: {analysis_capability.can_execute()}")

# Test capability execution
print("   Testing capability execution:")
result = analysis_capability.execute("Sample financial data for analysis", depth="advanced")
print(f"   Execution result: {result['success']}")
if result['success']:
    print(f"   Analysis insights: {result['result']['insights']}")
    print(f"   Execution time: {result['execution_time']:.3f}s")

# Test AgentTrait
print("\n🎭 Testing AgentTrait:")

analytical_trait = AgentTrait(
    trait_id="analytical_mindset",
    name="Analytical Mindset",
    description="Tendency to approach problems systematically and thoroughly",
    intensity=0.8,
    decision_modifier=0.2,
    speed_modifier=-0.1,  # Slightly slower due to thoroughness
    activation_contexts=["analysis", "research", "investigation"]
)

print(f"   Created trait: {analytical_trait.name}")
print(f"   Intensity: {analytical_trait.intensity}")

# Test trait activation
test_context = {"operation_type": "analysis", "urgency": "normal"}
is_active = analytical_trait.is_active(test_context)
print(f"   Active in analysis context: {is_active}")

# Test modifier application
base_speed = 0.8
modified_speed = analytical_trait.apply_modifiers(base_speed, "speed")
print(f"   Speed modification: {base_speed:.2f} → {modified_speed:.2f}")

# Test DomainKnowledge
print("\n📚 Testing DomainKnowledge:")

financial_knowledge = DomainKnowledge(AgentDomain.FINANCIAL)

# Add sample knowledge
financial_knowledge.add_fact(
    "roi_calculation",
    "ROI is calculated as (Gain - Cost) / Cost * 100",
    confidence=1.0,
    tags=["calculation", "metrics"]
)

financial_knowledge.add_rule(
    "risk_assessment",
    "If volatility > 0.3 and liquidity < 0.5",
    "Classify as high-risk investment",
    confidence=0.9
)

financial_knowledge.add_procedure(
    "due_diligence",
    "Investment Due Diligence",
    [
        "Review financial statements",
        "Analyze market position",
        "Assess management team",
        "Evaluate growth prospects",
        "Calculate valuation metrics"
    ]
)

financial_knowledge.add_terminology("P/E Ratio", "Price-to-Earnings ratio, a valuation metric")

# Test knowledge querying
print("   Testing knowledge query:")
query_results = financial_knowledge.query_knowledge("ROI calculation")
print(f"   Found {len(query_results)} results for 'ROI calculation'")
if query_results:
    print(f"   Top result: {query_results[0]['content']['description']}")

# Get knowledge statistics
stats = financial_knowledge.get_statistics()
print(f"   Knowledge base stats:")
print(f"     Total items: {stats['total_knowledge_items']}")
print(f"     Facts: {stats['facts_count']}")
print(f"     Rules: {stats['rules_count']}")
print(f"     Procedures: {stats['procedures_count']}")

# Test CustomAgentBase (with simple implementation)
print("\n🤖 Testing CustomAgentBase:")

class TestCustomAgent(CustomAgentBase):
    """Simple test implementation of CustomAgentBase"""

    def specialized_operation(self, operation_type: str, **kwargs) -> Dict[str, Any]:
        """Test implementation of specialized operation"""

        if operation_type == "domain_analysis":
            # Use domain knowledge
            query = kwargs.get('query', 'analysis')
            knowledge_results = self.domain_knowledge.query_knowledge(query)

            return {
                'success': True,
                'operation_type': operation_type,
                'query': query,
                'knowledge_results': len(knowledge_results),
                'agent_domain': self.domain.value
            }

        elif operation_type == "capability_assessment":
            return {
                'success': True,
                'operation_type': operation_type,
                'available_capabilities': self.get_available_capabilities(),
                'expertise_assessment': self.get_expertise_assessment()
            }

        else:
            return {
                'success': False,
                'error': f'Unknown operation type: {operation_type}'
            }

# Create test agent
test_agent = TestCustomAgent(
    name="FinancialTestBot",
    domain=AgentDomain.FINANCIAL
)

# Add capability and trait
test_agent.add_capability(analysis_capability)
test_agent.add_trait(analytical_trait)

# Update knowledge base
test_agent.domain_knowledge = financial_knowledge

# Test agent operations
print("   Testing agent operations:")

# Update context
test_agent.update_context({"operation_type": "analysis", "complexity": "high"})

# Execute capability
cap_result = test_agent.execute_capability("financial_analysis", "Portfolio data analysis")
print(f"   Capability execution: {cap_result['success']}")

# Execute specialized operation
spec_result = test_agent.specialized_operation("domain_analysis", query="investment")
print(f"   Specialized operation: {spec_result['success']}")
print(f"   Knowledge results found: {spec_result.get('knowledge_results', 0)}")

# Get expertise assessment
assessment = test_agent.get_expertise_assessment()
print(f"   Agent expertise score: {assessment['expertise_score']:.3f}")
print(f"   Success rate: {assessment['success_rate']:.3f}")
print(f"   Available capabilities: {len(assessment['available_capabilities'])}")

print()
print("🎉 Custom Agent Foundation working correctly!")
print("   Ready for Part 2: Specialized Agent Types")
print()
print("📋 Summary of what we built:")
print("   • AgentCapability - Executable skills with performance tracking")
print("   • AgentTrait - Behavioral modifiers for decision making")
print("   • DomainKnowledge - Expert knowledge base with querying")
print("   • CustomAgentBase - Foundation for specialized agents")
print("   • Capability execution with trait modifiers")
print("   • Performance assessment and expertise scoring")
print("   • Context-aware trait activation")
print("   • Comprehensive knowledge management")

🧪 Testing Custom Agent Foundation...

📋 Testing AgentCapability:
   Created capability: Financial Data Analysis
   Expertise level: ADVANCED
   Can execute: True
   Testing capability execution:
   Execution result: True
   Analysis insights: Analyzed 34 characters with advanced depth
   Execution time: 0.100s

🎭 Testing AgentTrait:
   Created trait: Analytical Mindset
   Intensity: 0.8
   Active in analysis context: False
   Speed modification: 0.80 → 0.72

📚 Testing DomainKnowledge:
📚 Domain knowledge initialized for: financial
   📝 Added fact: ROI is calculated as (Gain - Cost) / Cost * 100...
   📋 Added rule: If volatility > 0.3 and liquidity < 0.5 → Classify as high-risk investment
   📋 Added procedure: Investment Due Diligence (5 steps)
   📖 Added term: P/E Ratio
   Testing knowledge query:
   Found 0 results for 'ROI calculation'
   Knowledge base stats:
     Total items: 3
     Facts: 1
     Rules: 1
     Procedures: 1

🤖 Testing CustomAgentBase:
📚 Domain knowledge initialized 

In [4]:
"""
Tutorial 14 Part 2: Specialized Agent Types
==========================================

Building concrete specialized agent types for different domains
with specific capabilities, behaviors, and expert knowledge.
This part creates ready-to-use domain experts.

Run this after Part 1.
"""

import random
import math
from datetime import datetime, timedelta
from typing import List, Dict, Any, Optional
import json

print("Tutorial 14 Part 2: Specialized Agent Types")
print("=" * 44)
print("Building domain-specific expert agents...")
print()


Tutorial 14 Part 2: Specialized Agent Types
Building domain-specific expert agents...



In [7]:
# FINANCIAL ANALYST AGENT
# =============================================================================

class FinancialAnalystAgent(CustomAgentBase):
    """
    Specialized agent for financial analysis, portfolio management,
    and investment advisory services.

    This agent combines analytical capabilities with conservative
    decision-making traits and comprehensive financial knowledge.
    """

    def __init__(self, name: str = "FinancialAnalyst", risk_tolerance: str = "moderate"):
        super().__init__(name=name, domain=AgentDomain.FINANCIAL)

        self.risk_tolerance = risk_tolerance  # conservative, moderate, aggressive
        self.setup_financial_capabilities()
        self.setup_financial_traits()
        self.setup_financial_knowledge()

        print(f"💰 Financial Analyst Agent '{name}' initialized")
        print(f"   Risk tolerance: {risk_tolerance}")

    def setup_financial_capabilities(self):
        """Setup financial analysis capabilities"""

        # Portfolio Analysis Capability
        def portfolio_analysis(holdings: List[Dict], market_data: Dict = None):
            """Analyze a portfolio for risk, return, and diversification"""

            total_value = sum(holding.get('value', 0) for holding in holdings)

            if total_value == 0:
                return {'error': 'Empty portfolio'}

            # Calculate allocation percentages
            allocations = {
                holding['symbol']: holding['value'] / total_value
                for holding in holdings
            }

            # Simulate risk metrics
            portfolio_beta = sum(
                allocations[holding['symbol']] * holding.get('beta', 1.0)
                for holding in holdings
            )

            # Diversification score (simplified)
            diversification_score = min(1.0, len(holdings) / 10.0)

            # Risk assessment
            risk_level = "LOW" if portfolio_beta < 0.8 else "MODERATE" if portfolio_beta < 1.2 else "HIGH"

            return {
                'total_value': total_value,
                'holdings_count': len(holdings),
                'allocations': allocations,
                'portfolio_beta': portfolio_beta,
                'diversification_score': diversification_score,
                'risk_level': risk_level,
                'recommendations': self._generate_portfolio_recommendations(allocations, portfolio_beta)
            }

        portfolio_capability = AgentCapability(
            capability_id="portfolio_analysis",
            name="Portfolio Analysis",
            capability_type=CapabilityType.ANALYTICAL,
            expertise_level=ExpertiseLevel.ADVANCED,
            description="Comprehensive portfolio risk and return analysis",
            accuracy_rating=0.92,
            speed_rating=0.88,
            reliability_rating=0.95,
            execution_function=portfolio_analysis
        )
        self.add_capability(portfolio_capability)

        # Market Prediction Capability
        def market_prediction(symbol: str, time_horizon: str = "1_month", indicators: List[str] = None):
            """Predict market movements based on technical and fundamental analysis"""

            # Simulate market prediction logic
            base_confidence = 0.65

            # Adjust confidence based on time horizon
            horizon_adjustments = {
                "1_week": 0.15,
                "1_month": 0.0,
                "3_months": -0.10,
                "1_year": -0.20
            }

            confidence = base_confidence + horizon_adjustments.get(time_horizon, 0.0)

            # Simulate prediction direction
            directions = ["BULLISH", "BEARISH", "NEUTRAL"]
            weights = [0.4, 0.3, 0.3] if self.risk_tolerance == "aggressive" else [0.3, 0.3, 0.4]
            direction = random.choices(directions, weights=weights)[0]

            # Generate price targets
            current_price = random.uniform(50, 200)  # Simulated current price

            if direction == "BULLISH":
                target_price = current_price * random.uniform(1.05, 1.25)
            elif direction == "BEARISH":
                target_price = current_price * random.uniform(0.75, 0.95)
            else:
                target_price = current_price * random.uniform(0.95, 1.05)

            return {
                'symbol': symbol,
                'time_horizon': time_horizon,
                'current_price': current_price,
                'direction': direction,
                'target_price': target_price,
                'confidence': confidence,
                'potential_return': (target_price - current_price) / current_price,
                'risk_factors': self._identify_risk_factors(symbol, direction)
            }

        prediction_capability = AgentCapability(
            capability_id="market_prediction",
            name="Market Prediction",
            capability_type=CapabilityType.ANALYTICAL,
            expertise_level=ExpertiseLevel.ADVANCED,
            description="Market movement prediction with risk assessment",
            accuracy_rating=0.68,  # Realistic for market prediction
            speed_rating=0.85,
            reliability_rating=0.80,
            cooldown_seconds=5.0,  # Limit frequent predictions
            execution_function=market_prediction
        )
        self.add_capability(prediction_capability)

        # Risk Assessment Capability
        def risk_assessment(investment_data: Dict):
            """Assess investment risk across multiple dimensions"""

            risk_factors = {
                'market_risk': random.uniform(0.2, 0.8),
                'credit_risk': random.uniform(0.1, 0.6),
                'liquidity_risk': random.uniform(0.1, 0.5),
                'operational_risk': random.uniform(0.05, 0.3),
                'regulatory_risk': random.uniform(0.1, 0.4)
            }

            # Calculate overall risk score
            weights = {'market_risk': 0.3, 'credit_risk': 0.25, 'liquidity_risk': 0.2,
                      'operational_risk': 0.15, 'regulatory_risk': 0.1}

            overall_risk = sum(risk_factors[factor] * weight for factor, weight in weights.items())

            # Risk categorization
            if overall_risk < 0.3:
                risk_category = "LOW"
            elif overall_risk < 0.6:
                risk_category = "MODERATE"
            else:
                risk_category = "HIGH"

            return {
                'overall_risk_score': overall_risk,
                'risk_category': risk_category,
                'risk_breakdown': risk_factors,
                'risk_mitigation_strategies': self._suggest_risk_mitigation(risk_factors),
                'suitability_assessment': self._assess_suitability(overall_risk)
            }

        risk_capability = AgentCapability(
            capability_id="risk_assessment",
            name="Investment Risk Assessment",
            capability_type=CapabilityType.ANALYTICAL,
            expertise_level=ExpertiseLevel.MASTER,
            description="Multi-dimensional investment risk analysis",
            accuracy_rating=0.90,
            speed_rating=0.92,
            reliability_rating=0.94,
            execution_function=risk_assessment
        )
        self.add_capability(risk_capability)

    def setup_financial_traits(self):
        """Setup behavioral traits for financial analysis"""

        # Conservative Decision Making
        conservative_trait = AgentTrait(
            trait_id="conservative_analysis",
            name="Conservative Analysis",
            description="Tendency to prioritize capital preservation over aggressive growth",
            intensity=0.8 if self.risk_tolerance == "conservative" else 0.4,
            decision_modifier=-0.2,  # More cautious decisions
            risk_modifier=-0.3,     # Lower risk tolerance
            speed_modifier=-0.1,    # More thorough analysis
            activation_contexts=["investment_analysis", "portfolio_review", "risk_assessment"]
        )
        self.add_trait(conservative_trait)

        # Detail-Oriented Trait
        detail_oriented_trait = AgentTrait(
            trait_id="detail_oriented",
            name="Detail-Oriented Analysis",
            description="Focus on comprehensive analysis and due diligence",
            intensity=0.9,
            decision_modifier=0.15,   # More thorough decisions
            speed_modifier=-0.2,     # Slower but more thorough
            activation_contexts=["financial_analysis", "due_diligence", "report_generation"]
        )
        self.add_trait(detail_oriented_trait)

        # Market Awareness
        market_awareness_trait = AgentTrait(
            trait_id="market_awareness",
            name="Market Awareness",
            description="Strong focus on market conditions and economic indicators",
            intensity=0.85,
            decision_modifier=0.1,
            communication_modifier=0.2,  # More informative communication
            activation_contexts=["market_analysis", "economic_review", "trend_analysis"]
        )
        self.add_trait(market_awareness_trait)

    def setup_financial_knowledge(self):
        """Setup comprehensive financial knowledge base"""

        # Add fundamental financial facts
        self.domain_knowledge.add_fact(
            "compound_interest",
            "Compound interest is interest calculated on the initial principal and accumulated interest",
            confidence=1.0,
            tags=["fundamental", "calculation", "investment"]
        )

        self.domain_knowledge.add_fact(
            "diversification_principle",
            "Diversification reduces risk by spreading investments across different assets, sectors, and geographies",
            confidence=1.0,
            tags=["risk_management", "portfolio", "strategy"]
        )

        self.domain_knowledge.add_fact(
            "risk_return_relationship",
            "Higher potential returns generally come with higher risk - the fundamental risk-return tradeoff",
            confidence=1.0,
            tags=["risk", "return", "principle"]
        )

        # Add financial rules
        self.domain_knowledge.add_rule(
            "portfolio_allocation_rule",
            "If investor age > 30 and risk_tolerance = conservative",
            "Allocate maximum 60% to equities, 40% to bonds and cash",
            confidence=0.85
        )

        self.domain_knowledge.add_rule(
            "emergency_fund_rule",
            "If emergency_fund < 6_months_expenses",
            "Prioritize building emergency fund before aggressive investing",
            confidence=0.95
        )

        self.domain_knowledge.add_rule(
            "rebalancing_rule",
            "If any asset allocation deviates > 5% from target",
            "Recommend portfolio rebalancing to maintain target allocation",
            confidence=0.90
        )

        # Add investment procedures
        self.domain_knowledge.add_procedure(
            "investment_analysis",
            "Complete Investment Analysis",
            [
                "Define investment objectives and constraints",
                "Analyze current financial position",
                "Evaluate investment options",
                "Assess risk-return profiles",
                "Perform sensitivity analysis",
                "Make investment recommendation",
                "Develop monitoring plan"
            ],
            prerequisites=["financial_data", "risk_profile", "investment_objectives"]
        )

        self.domain_knowledge.add_procedure(
            "portfolio_construction",
            "Portfolio Construction Process",
            [
                "Determine strategic asset allocation",
                "Select specific investments within each asset class",
                "Optimize for risk-adjusted returns",
                "Consider tax implications",
                "Implement position sizing",
                "Set up monitoring and rebalancing schedule"
            ]
        )

        # Add financial terminology
        financial_terms = {
            "Alpha": "A measure of investment performance relative to a benchmark",
            "Beta": "A measure of investment volatility relative to the market",
            "Sharpe Ratio": "Risk-adjusted return metric (excess return per unit of risk)",
            "P/E Ratio": "Price-to-Earnings ratio, a valuation metric",
            "ROE": "Return on Equity, measuring company profitability efficiency",
            "EBITDA": "Earnings Before Interest, Taxes, Depreciation, and Amortization",
            "Duration": "Measure of interest rate sensitivity for bonds",
            "Volatility": "Statistical measure of price variation over time"
        }

        for term, definition in financial_terms.items():
            self.domain_knowledge.add_terminology(term, definition)

    def specialized_operation(self, operation_type: str, **kwargs) -> Dict[str, Any]:
        """Financial analyst specialized operations"""

        if operation_type == "complete_portfolio_analysis":
            return self._complete_portfolio_analysis(**kwargs)

        elif operation_type == "investment_recommendation":
            return self._investment_recommendation(**kwargs)

        elif operation_type == "market_outlook":
            return self._market_outlook(**kwargs)

        elif operation_type == "financial_planning":
            return self._financial_planning(**kwargs)

        else:
            return {
                'success': False,
                'error': f'Unknown financial operation: {operation_type}',
                'available_operations': [
                    'complete_portfolio_analysis',
                    'investment_recommendation',
                    'market_outlook',
                    'financial_planning'
                ]
            }

    def _complete_portfolio_analysis(self, portfolio_data: Dict, **kwargs) -> Dict[str, Any]:
        """Comprehensive portfolio analysis combining multiple capabilities"""

        # Execute portfolio analysis capability
        portfolio_result = self.execute_capability(
            "portfolio_analysis",
            portfolio_data.get('holdings', []),
            portfolio_data.get('market_data', {})
        )

        # Execute risk assessment
        risk_result = self.execute_capability(
            "risk_assessment",
            portfolio_data
        )

        # Generate comprehensive report
        analysis_report = {
            'analysis_date': datetime.now().isoformat(),
            'portfolio_summary': portfolio_result.get('result', {}),
            'risk_analysis': risk_result.get('result', {}),
            'recommendations': self._generate_comprehensive_recommendations(
                portfolio_result.get('result', {}),
                risk_result.get('result', {})
            ),
            'next_review_date': (datetime.now() + timedelta(days=90)).isoformat(),
            'analyst_confidence': min(
                portfolio_result.get('result', {}).get('confidence', 0.8),
                risk_result.get('result', {}).get('confidence', 0.8)
            )
        }

        # Generate executive summary
        analysis_report['executive_summary'] = self._generate_executive_summary(analysis_report)

        return {
            'success': True,
            'operation_type': 'complete_portfolio_analysis',
            'analysis_report': analysis_report,
            'executive_summary': self._generate_executive_summary(analysis_report)
        }

    def _investment_recommendation(self, investment_criteria: Dict, **kwargs) -> Dict[str, Any]:
        """Generate investment recommendations based on criteria"""

        # Analyze investment universe (simulated)
        investment_options = [
            {'symbol': 'SPY', 'type': 'ETF', 'sector': 'Broad Market', 'risk': 'Moderate'},
            {'symbol': 'QQQ', 'type': 'ETF', 'sector': 'Technology', 'risk': 'High'},
            {'symbol': 'BND', 'type': 'Bond ETF', 'sector': 'Fixed Income', 'risk': 'Low'},
            {'symbol': 'VTI', 'type': 'ETF', 'sector': 'Total Market', 'risk': 'Moderate'},
            {'symbol': 'VXUS', 'type': 'ETF', 'sector': 'International', 'risk': 'Moderate'}
        ]

        # Filter based on risk tolerance
        suitable_options = [
            option for option in investment_options
            if self._is_suitable_investment(option, investment_criteria)
        ]

        # Generate predictions for suitable options
        recommendations = []
        for option in suitable_options[:3]:  # Top 3 recommendations
            prediction = self.execute_capability(
                "market_prediction",
                option['symbol'],
                investment_criteria.get('time_horizon', '1_month')
            )

            if prediction.get('success'):
                recommendations.append({
                    'investment': option,
                    'prediction': prediction['result'],
                    'allocation_percentage': self._calculate_allocation(option, investment_criteria),
                    'rationale': self._generate_investment_rationale(option, prediction['result'])
                })

        return {
            'success': True,
            'operation_type': 'investment_recommendation',
            'recommendations': recommendations,
            'criteria_used': investment_criteria,
            'disclaimer': "Past performance does not guarantee future results. All investments carry risk."
        }

    def _market_outlook(self, market_scope: str = "US_EQUITY", **kwargs) -> Dict[str, Any]:
        """Generate market outlook and analysis"""

        # Generate market predictions for key indices
        key_markets = {
            'US_EQUITY': ['SPY', 'QQQ', 'IWM'],
            'INTERNATIONAL': ['EFA', 'EEM', 'VWO'],
            'FIXED_INCOME': ['BND', 'TLT', 'HYG'],
            'COMMODITIES': ['GLD', 'USO', 'DBA']
        }

        markets_to_analyze = key_markets.get(market_scope, key_markets['US_EQUITY'])

        market_predictions = {}
        for symbol in markets_to_analyze:
            prediction = self.execute_capability(
                "market_prediction",
                symbol,
                kwargs.get('time_horizon', '3_months')
            )

            if prediction.get('success'):
                market_predictions[symbol] = prediction['result']

        # Generate overall market sentiment
        bullish_count = sum(1 for pred in market_predictions.values() if pred['direction'] == 'BULLISH')
        bearish_count = sum(1 for pred in market_predictions.values() if pred['direction'] == 'BEARISH')

        if bullish_count > bearish_count:
            overall_sentiment = "OPTIMISTIC"
        elif bearish_count > bullish_count:
            overall_sentiment = "CAUTIOUS"
        else:
            overall_sentiment = "NEUTRAL"

        return {
            'success': True,
            'operation_type': 'market_outlook',
            'market_scope': market_scope,
            'overall_sentiment': overall_sentiment,
            'individual_predictions': market_predictions,
            'key_themes': self._identify_market_themes(market_predictions),
            'risk_factors': self._identify_market_risks(market_predictions),
            'investment_implications': self._generate_investment_implications(overall_sentiment)
        }

    def _financial_planning(self, client_profile: Dict, **kwargs) -> Dict[str, Any]:
        """Comprehensive financial planning analysis"""

        # Extract client information
        age = client_profile.get('age', 35)
        income = client_profile.get('annual_income', 75000)
        goals = client_profile.get('goals', ['retirement', 'emergency_fund'])
        risk_tolerance = client_profile.get('risk_tolerance', 'moderate')

        # Calculate planning metrics
        retirement_goal = income * 25  # Simple 25x rule
        emergency_fund_target = income * 0.5  # 6 months expenses (assuming 50% of income)

        # Generate asset allocation recommendation
        if age < 30:
            equity_allocation = 0.9
        elif age < 50:
            equity_allocation = 0.7
        else:
            equity_allocation = 0.5

        # Adjust for risk tolerance
        risk_adjustments = {'conservative': -0.2, 'moderate': 0.0, 'aggressive': 0.2}
        equity_allocation += risk_adjustments.get(risk_tolerance, 0.0)
        equity_allocation = max(0.3, min(0.9, equity_allocation))  # Keep within bounds

        planning_recommendations = {
            'asset_allocation': {
                'equities': equity_allocation,
                'bonds': 1 - equity_allocation,
                'cash': 0.05
            },
            'savings_targets': {
                'emergency_fund': emergency_fund_target,
                'retirement_goal': retirement_goal,
                'monthly_savings_needed': retirement_goal / ((65 - age) * 12)
            },
            'priority_goals': self._prioritize_goals(goals, client_profile),
            'action_items': self._generate_action_items(client_profile)
        }

        return {
            'success': True,
            'operation_type': 'financial_planning',
            'client_profile': client_profile,
            'planning_recommendations': planning_recommendations,
            'timeline': f"{65 - age} years to retirement",
            'next_review_period': "Annual review recommended"
        }

    # Helper methods
    def _generate_portfolio_recommendations(self, allocations: Dict, portfolio_beta: float) -> List[str]:
        """Generate portfolio improvement recommendations"""
        recommendations = []

        # Check for concentration risk
        max_allocation = max(allocations.values()) if allocations else 0
        if max_allocation > 0.3:
            recommendations.append("Consider reducing concentration - largest holding exceeds 30%")

        # Check beta level
        if portfolio_beta > 1.5:
            recommendations.append("Portfolio is highly volatile - consider adding defensive positions")
        elif portfolio_beta < 0.5:
            recommendations.append("Portfolio may be too conservative - consider growth opportunities")

        # Check diversification
        if len(allocations) < 5:
            recommendations.append("Increase diversification by adding more holdings")

        return recommendations

    def _identify_risk_factors(self, symbol: str, direction: str) -> List[str]:
        """Identify risk factors for market prediction"""
        base_risks = ["Market volatility", "Economic uncertainty", "Regulatory changes"]

        if direction == "BULLISH":
            base_risks.extend(["Overvaluation risk", "Momentum reversal"])
        elif direction == "BEARISH":
            base_risks.extend(["Overselling risk", "Support level breaks"])

        return base_risks

    def _suggest_risk_mitigation(self, risk_factors: Dict) -> List[str]:
        """Suggest risk mitigation strategies"""
        strategies = []

        if risk_factors.get('market_risk', 0) > 0.6:
            strategies.append("Consider hedging with defensive positions")

        if risk_factors.get('liquidity_risk', 0) > 0.5:
            strategies.append("Maintain adequate cash reserves")

        if risk_factors.get('credit_risk', 0) > 0.5:
            strategies.append("Focus on investment-grade securities")

        return strategies

    def _assess_suitability(self, overall_risk: float) -> str:
        """Assess investment suitability based on risk"""
        tolerance_map = {
            'conservative': 0.4,
            'moderate': 0.6,
            'aggressive': 0.8
        }

        max_acceptable_risk = tolerance_map.get(self.risk_tolerance, 0.6)

        if overall_risk <= max_acceptable_risk:
            return "SUITABLE"
        elif overall_risk <= max_acceptable_risk + 0.1:
            return "MARGINAL"
        else:
            return "UNSUITABLE"

    def _generate_comprehensive_recommendations(self, portfolio_analysis: Dict, risk_analysis: Dict) -> List[str]:
        """Generate comprehensive recommendations combining multiple analyses"""
        recommendations = []

        # Portfolio-based recommendations
        if portfolio_analysis.get('risk_level') == 'HIGH' and risk_analysis.get('risk_category') == 'HIGH':
            recommendations.append("PRIORITY: Reduce portfolio risk through diversification and defensive positions")

        # Risk-based recommendations
        overall_risk = risk_analysis.get('overall_risk_score', 0.5)
        if overall_risk > 0.7:
            recommendations.append("Consider reducing position sizes and increasing cash allocation")

        return recommendations

    def _generate_executive_summary(self, analysis_report: Dict) -> str:
        """Generate executive summary of analysis"""
        portfolio_summary = analysis_report.get('portfolio_summary', {})
        risk_analysis = analysis_report.get('risk_analysis', {})

        summary = f"Portfolio Analysis Summary:\n"
        summary += f"• Total Value: ${portfolio_summary.get('total_value', 0):,.2f}\n"
        summary += f"• Risk Level: {portfolio_summary.get('risk_level', 'Unknown')}\n"
        summary += f"• Diversification Score: {portfolio_summary.get('diversification_score', 0):.2f}\n"
        summary += f"• Overall Risk Category: {risk_analysis.get('risk_category', 'Unknown')}\n"

        return summary

    def _is_suitable_investment(self, investment: Dict, criteria: Dict) -> bool:
        """Check if investment meets criteria"""
        client_risk_tolerance = criteria.get('risk_tolerance', 'moderate')

        risk_compatibility = {
            'conservative': ['Low'],
            'moderate': ['Low', 'Moderate'],
            'aggressive': ['Low', 'Moderate', 'High']
        }

        return investment['risk'] in risk_compatibility.get(client_risk_tolerance, ['Low'])

    def _calculate_allocation(self, investment: Dict, criteria: Dict) -> float:
        """Calculate recommended allocation percentage"""
        base_allocation = {
            'Low': 0.4,
            'Moderate': 0.3,
            'High': 0.2
        }

        return base_allocation.get(investment['risk'], 0.2)

    def _generate_investment_rationale(self, investment: Dict, prediction: Dict) -> str:
        """Generate rationale for investment recommendation"""
        direction = prediction.get('direction', 'NEUTRAL')
        confidence = prediction.get('confidence', 0.5)

        rationale = f"Recommended due to {direction.lower()} outlook "
        rationale += f"with {confidence:.0%} confidence. "
        rationale += f"Suitable for {investment['risk'].lower()} risk tolerance."

        return rationale

    def _identify_market_themes(self, predictions: Dict) -> List[str]:
        """Identify key market themes from predictions"""
        themes = []

        bullish_count = sum(1 for pred in predictions.values() if pred['direction'] == 'BULLISH')
        total_predictions = len(predictions)

        if bullish_count / total_predictions > 0.6:
            themes.append("Risk-on sentiment across markets")
        elif bullish_count / total_predictions < 0.4:
            themes.append("Risk-off sentiment and defensive positioning")
        else:
            themes.append("Mixed market signals and selective opportunities")

        return themes

    def _identify_market_risks(self, predictions: Dict) -> List[str]:
        """Identify market risks from predictions"""
        risks = ["Monetary policy uncertainty", "Geopolitical tensions", "Economic data volatility"]

        # Add specific risks based on predictions
        low_confidence_count = sum(1 for pred in predictions.values() if pred['confidence'] < 0.6)
        if low_confidence_count > len(predictions) / 2:
            risks.append("High market uncertainty and unpredictability")

        return risks

    def _generate_investment_implications(self, sentiment: str) -> List[str]:
        """Generate investment implications based on sentiment"""
        implications = {
            'OPTIMISTIC': [
                "Consider increasing equity allocation",
                "Focus on growth-oriented investments",
                "May be good time for tactical overweights"
            ],
            'CAUTIOUS': [
                "Maintain defensive positioning",
                "Increase cash and bond allocation",
                "Focus on quality and dividend-paying stocks"
            ],
            'NEUTRAL': [
                "Maintain strategic asset allocation",
                "Focus on rebalancing opportunities",
                "Consider value opportunities"
            ]
        }

        return implications.get(sentiment, ["Maintain current positioning"])

    def _prioritize_goals(self, goals: List[str], profile: Dict) -> List[Dict[str, Any]]:
        """Prioritize financial goals based on profile"""
        goal_priorities = {
            'emergency_fund': {'priority': 1, 'timeline': 'Immediate'},
            'retirement': {'priority': 2, 'timeline': 'Long-term'},
            'house_purchase': {'priority': 3, 'timeline': 'Medium-term'},
            'education': {'priority': 4, 'timeline': 'Medium-term'}
        }

        prioritized_goals = []
        for goal in goals:
            if goal in goal_priorities:
                goal_info = goal_priorities[goal].copy()
                goal_info['goal'] = goal
                prioritized_goals.append(goal_info)

        return sorted(prioritized_goals, key=lambda x: x['priority'])

    def _generate_action_items(self, profile: Dict) -> List[str]:
        """Generate specific action items for client"""
        actions = [
            "Review and update investment policy statement",
            "Rebalance portfolio to target allocation",
            "Set up automatic savings transfers"
        ]

        if profile.get('age', 35) > 50:
            actions.append("Consider catch-up retirement contributions")

        return actions

# =============================================================================
# HEALTHCARE ADVISOR AGENT
# =============================================================================

class HealthcareAdvisorAgent(CustomAgentBase):
    """
    Specialized agent for healthcare guidance, symptom analysis,
    and medical information provision.

    This agent prioritizes safety and provides medically-informed
    guidance with appropriate disclaimers.
    """

    def __init__(self, name: str = "HealthcareAdvisor", specialty: str = "general"):
        super().__init__(name=name, domain=AgentDomain.HEALTHCARE)

        self.specialty = specialty  # general, cardiology, pediatrics, etc.
        self.setup_healthcare_capabilities()
        self.setup_healthcare_traits()
        self.setup_healthcare_knowledge()

        print(f"🏥 Healthcare Advisor Agent '{name}' initialized")
        print(f"   Specialty: {specialty}")
        print(f"   ⚠️  Medical disclaimer: Always consult healthcare professionals for medical advice")

    def setup_healthcare_capabilities(self):
        """Setup healthcare analysis capabilities"""

        # Symptom Analysis Capability
        def symptom_analysis(symptoms: List[str], patient_info: Dict = None):
            """Analyze symptoms and provide general health guidance"""

            if not symptoms:
                return {'error': 'No symptoms provided for analysis'}

            patient_info = patient_info or {}
            age = patient_info.get('age', 'unknown')
            gender = patient_info.get('gender', 'unknown')

            # Categorize symptoms by severity (simplified logic)
            emergency_keywords = ['chest pain', 'difficulty breathing', 'severe bleeding',
                                'loss of consciousness', 'severe allergic reaction']
            urgent_keywords = ['high fever', 'persistent vomiting', 'severe pain',
                             'vision changes', 'confusion']

            severity_level = "ROUTINE"
            for symptom in symptoms:
                symptom_lower = symptom.lower()
                if any(keyword in symptom_lower for keyword in emergency_keywords):
                    severity_level = "EMERGENCY"
                    break
                elif any(keyword in symptom_lower for keyword in urgent_keywords):
                    severity_level = "URGENT"

            # Generate possible conditions (educational purposes)
            possible_conditions = self._generate_possible_conditions(symptoms)

            # Generate recommendations
            recommendations = self._generate_health_recommendations(symptoms, severity_level)

            return {
                'symptoms_analyzed': symptoms,
                'severity_level': severity_level,
                'possible_conditions': possible_conditions,
                'recommendations': recommendations,
                'when_to_seek_care': self._get_care_urgency_guidance(severity_level),
                'self_care_options': self._get_self_care_options(symptoms),
                'red_flags': self._identify_red_flags(symptoms),
                'disclaimer': "This is for educational purposes only. Always consult a healthcare professional for medical advice."
            }

        symptom_capability = AgentCapability(
            capability_id="symptom_analysis",
            name="Symptom Analysis",
            capability_type=CapabilityType.ANALYTICAL,
            expertise_level=ExpertiseLevel.ADVANCED,
            description="Educational symptom analysis with safety recommendations",
            accuracy_rating=0.75,  # Conservative for medical information
            speed_rating=0.90,
            reliability_rating=0.95,
            execution_function=symptom_analysis
        )
        self.add_capability(symptom_capability)

        # Health Information Capability
        def health_information(topic: str, detail_level: str = "general"):
            """Provide educational health information on medical topics"""

            # Simulate health information database
            health_topics = {
                'diabetes': {
                    'definition': 'A group of metabolic disorders characterized by high blood sugar',
                    'types': ['Type 1', 'Type 2', 'Gestational'],
                    'symptoms': ['Increased thirst', 'Frequent urination', 'Fatigue', 'Blurred vision'],
                    'management': ['Blood sugar monitoring', 'Medication adherence', 'Diet management', 'Regular exercise'],
                    'complications': ['Heart disease', 'Kidney disease', 'Eye problems', 'Nerve damage']
                },
                'hypertension': {
                    'definition': 'High blood pressure - force of blood against artery walls is too high',
                    'types': ['Primary', 'Secondary'],
                    'symptoms': ['Often no symptoms', 'Headaches', 'Shortness of breath', 'Nosebleeds'],
                    'management': ['Lifestyle changes', 'Medication', 'Regular monitoring', 'Stress management'],
                    'complications': ['Heart attack', 'Stroke', 'Kidney disease', 'Heart failure']
                },
                'nutrition': {
                    'definition': 'The process of providing or obtaining food necessary for health and growth',
                    'principles': ['Balanced diet', 'Portion control', 'Variety', 'Moderation'],
                    'guidelines': ['5+ servings fruits/vegetables daily', 'Whole grains', 'Lean proteins', 'Limited processed foods'],
                    'benefits': ['Disease prevention', 'Energy maintenance', 'Mental health', 'Longevity']
                }
            }

            topic_info = health_topics.get(topic.lower())
            if not topic_info:
                return {
                    'error': f'Topic "{topic}" not found in health information database',
                    'available_topics': list(health_topics.keys())
                }

            # Adjust detail level
            if detail_level == "basic":
                return {
                    'topic': topic,
                    'definition': topic_info['definition'],
                    'key_points': list(topic_info.values())[1] if len(topic_info) > 1 else []
                }
            else:
                return {
                    'topic': topic,
                    'comprehensive_info': topic_info,
                    'additional_resources': 'Consult healthcare providers or reputable medical websites for more information'
                }

        health_info_capability = AgentCapability(
            capability_id="health_information",
            name="Health Information Provider",
            capability_type=CapabilityType.ADVISORY,
            expertise_level=ExpertiseLevel.ADVANCED,
            description="Educational health information on medical topics",
            accuracy_rating=0.90,
            speed_rating=0.95,
            reliability_rating=0.98,
            execution_function=health_information
        )
        self.add_capability(health_info_capability)

        # Emergency Triage Capability
        def emergency_triage(situation: str, symptoms: List[str], vital_signs: Dict = None):
            """Provide emergency triage guidance for urgent situations"""

            # Emergency indicators
            critical_indicators = [
                'unconscious', 'not breathing', 'no pulse', 'severe bleeding',
                'chest pain with shortness of breath', 'severe allergic reaction',
                'stroke symptoms', 'severe burns'
            ]

            # Check for critical conditions
            is_critical = any(
                indicator in situation.lower() or
                any(indicator in symptom.lower() for symptom in symptoms)
                for indicator in critical_indicators
            )

            if is_critical:
                priority_level = "CRITICAL - CALL 911 IMMEDIATELY"
                actions = [
                    "Call emergency services (911) immediately",
                    "Do not delay for any reason",
                    "Follow dispatcher instructions",
                    "Prepare for emergency responders"
                ]
            else:
                # Assess urgency level
                urgent_indicators = [
                    'high fever', 'severe pain', 'persistent vomiting',
                    'difficulty breathing', 'confusion', 'severe headache'
                ]

                is_urgent = any(
                    indicator in situation.lower() or
                    any(indicator in symptom.lower() for symptom in symptoms)
                    for indicator in urgent_indicators
                )

                if is_urgent:
                    priority_level = "URGENT - Seek immediate medical care"
                    actions = [
                        "Go to emergency room or urgent care",
                        "Call healthcare provider immediately",
                        "Do not wait for appointment",
                        "Monitor symptoms closely"
                    ]
                else:
                    priority_level = "MODERATE - Schedule medical appointment"
                    actions = [
                        "Contact healthcare provider within 24 hours",
                        "Monitor symptoms for changes",
                        "Use self-care measures as appropriate",
                        "Seek immediate care if symptoms worsen"
                    ]

            return {
                'situation_assessment': situation,
                'priority_level': priority_level,
                'immediate_actions': actions,
                'monitoring_instructions': self._get_monitoring_instructions(symptoms),
                'when_to_escalate': self._get_escalation_criteria(),
                'emergency_contacts': {
                    'emergency_services': '911',
                    'poison_control': '1-800-222-1222',
                    'crisis_hotline': '988'
                },
                'critical_disclaimer': "In life-threatening emergencies, call 911 immediately. This guidance is educational only."
            }

        triage_capability = AgentCapability(
            capability_id="emergency_triage",
            name="Emergency Triage Assessment",
            capability_type=CapabilityType.ADVISORY,
            expertise_level=ExpertiseLevel.MASTER,
            description="Emergency situation assessment and triage guidance",
            accuracy_rating=0.95,  # High accuracy for safety
            speed_rating=0.98,     # Must be very fast for emergencies
            reliability_rating=0.99,
            execution_function=emergency_triage
        )
        self.add_capability(triage_capability)

    def setup_healthcare_traits(self):
        """Setup healthcare-specific behavioral traits"""

        # Safety-First Trait
        safety_first_trait = AgentTrait(
            trait_id="safety_first",
            name="Safety-First Approach",
            description="Prioritizes patient safety and conservative recommendations",
            intensity=0.95,
            decision_modifier=0.3,   # More conservative decisions
            risk_modifier=-0.4,     # Very low risk tolerance
            communication_modifier=0.2,  # Clear, careful communication
            activation_contexts=["medical_advice", "symptom_analysis", "emergency_triage"]
        )
        self.add_trait(safety_first_trait)

        # Empathetic Communication
        empathetic_trait = AgentTrait(
            trait_id="empathetic_communication",
            name="Empathetic Communication",
            description="Communicates with empathy and understanding for health concerns",
            intensity=0.85,
            communication_modifier=0.3,
            decision_modifier=0.1,
            activation_contexts=["patient_interaction", "health_counseling", "emotional_support"]
        )
        self.add_trait(empathetic_trait)

        # Evidence-Based Practice
        evidence_based_trait = AgentTrait(
            trait_id="evidence_based",
            name="Evidence-Based Practice",
            description="Relies on scientific evidence and medical best practices",
            intensity=0.9,
            decision_modifier=0.2,
            speed_modifier=-0.1,  # Takes time to consider evidence
            activation_contexts=["medical_information", "treatment_recommendations", "research_based_advice"]
        )
        self.add_trait(evidence_based_trait)

    def setup_healthcare_knowledge(self):
        """Setup comprehensive healthcare knowledge base"""

        # Add medical facts
        self.domain_knowledge.add_fact(
            "normal_vital_signs",
            "Normal adult vital signs: BP <120/80, HR 60-100 bpm, RR 12-20/min, Temp 97-99°F",
            confidence=1.0,
            tags=["vital_signs", "normal_values", "assessment"]
        )

        self.domain_knowledge.add_fact(
            "hand_hygiene",
            "Hand hygiene is the most effective way to prevent infection transmission",
            confidence=1.0,
            tags=["infection_control", "prevention", "basic_care"]
        )

        self.domain_knowledge.add_fact(
            "medication_adherence",
            "Medication adherence improves outcomes - take medications as prescribed",
            confidence=0.95,
            tags=["medication", "compliance", "outcomes"]
        )

        # Add healthcare rules
        self.domain_knowledge.add_rule(
            "chest_pain_rule",
            "If patient reports chest pain with shortness of breath",
            "Recommend immediate emergency medical evaluation",
            confidence=0.98
        )

        self.domain_knowledge.add_rule(
            "fever_in_infants",
            "If infant under 3 months has fever >100.4°F",
            "Recommend immediate medical evaluation",
            confidence=0.99
        )

        self.domain_knowledge.add_rule(
            "medication_interactions",
            "If patient takes multiple medications",
            "Recommend consulting pharmacist or physician about interactions",
            confidence=0.90
        )

        # Add healthcare procedures
        self.domain_knowledge.add_procedure(
            "basic_first_aid",
            "Basic First Aid Assessment",
            [
                "Ensure scene safety",
                "Check for responsiveness",
                "Call for help if needed",
                "Assess airway, breathing, circulation",
                "Provide appropriate first aid",
                "Monitor until help arrives"
            ],
            prerequisites=["first_aid_training"],
            expected_outcome="Stabilized patient until professional help arrives"
        )

        self.domain_knowledge.add_procedure(
            "medication_review",
            "Comprehensive Medication Review",
            [
                "List all current medications",
                "Check for duplications",
                "Identify potential interactions",
                "Review dosing appropriateness",
                "Assess adherence patterns",
                "Provide recommendations"
            ]
        )

        # Add medical terminology
        medical_terms = {
            "Hypertension": "High blood pressure - sustained elevation of blood pressure",
            "Diabetes": "Group of metabolic disorders characterized by high blood sugar",
            "Myocardial Infarction": "Heart attack - death of heart muscle due to lack of blood supply",
            "Pneumonia": "Infection that inflames air sacs in one or both lungs",
            "Anemia": "Condition where blood lacks adequate healthy red blood cells",
            "Tachycardia": "Rapid heart rate - typically over 100 beats per minute",
            "Bradycardia": "Slow heart rate - typically under 60 beats per minute",
            "Dyspnea": "Shortness of breath or difficulty breathing"
        }

        for term, definition in medical_terms.items():
            self.domain_knowledge.add_terminology(term, definition)

    def specialized_operation(self, operation_type: str, **kwargs) -> Dict[str, Any]:
        """Healthcare advisor specialized operations"""

        if operation_type == "comprehensive_health_assessment":
            return self._comprehensive_health_assessment(**kwargs)

        elif operation_type == "medication_consultation":
            return self._medication_consultation(**kwargs)

        elif operation_type == "wellness_planning":
            return self._wellness_planning(**kwargs)

        elif operation_type == "health_education":
            return self._health_education(**kwargs)

        else:
            return {
                'success': False,
                'error': f'Unknown healthcare operation: {operation_type}',
                'available_operations': [
                    'comprehensive_health_assessment',
                    'medication_consultation',
                    'wellness_planning',
                    'health_education'
                ]
            }

    def _comprehensive_health_assessment(self, health_data: Dict, **kwargs) -> Dict[str, Any]:
        """Comprehensive health assessment combining multiple capabilities"""

        symptoms = health_data.get('symptoms', [])
        patient_info = health_data.get('patient_info', {})

        # Perform symptom analysis
        symptom_result = self.execute_capability("symptom_analysis", symptoms, patient_info)

        # Check if emergency triage needed
        if symptom_result.get('result', {}).get('severity_level') in ['EMERGENCY', 'URGENT']:
            triage_result = self.execute_capability(
                "emergency_triage",
                f"Patient presenting with: {', '.join(symptoms)}",
                symptoms,
                health_data.get('vital_signs', {})
            )
        else:
            triage_result = {'result': {'priority_level': 'ROUTINE'}}

        # Generate comprehensive assessment
        assessment = {
            'assessment_date': datetime.now().isoformat(),
            'patient_summary': patient_info,
            'symptom_analysis': symptom_result.get('result', {}),
            'triage_assessment': triage_result.get('result', {}),
            'health_recommendations': self._generate_health_action_plan(
                symptom_result.get('result', {}),
                triage_result.get('result', {})
            ),
            'follow_up_plan': self._create_follow_up_plan(symptom_result.get('result', {})),
            'educational_resources': self._recommend_educational_resources(symptoms)
        }

        return {
            'success': True,
            'operation_type': 'comprehensive_health_assessment',
            'health_assessment': assessment,
            'safety_disclaimer': "This assessment is for educational purposes only. Always consult healthcare professionals for medical decisions.",
            'emergency_note': "If this is a medical emergency, call 911 immediately."
        }

    def _medication_consultation(self, medication_data: Dict, **kwargs) -> Dict[str, Any]:
        """Provide medication information and interaction checking"""

        medications = medication_data.get('medications', [])
        new_medication = medication_data.get('new_medication')
        patient_conditions = medication_data.get('conditions', [])

        # Simulate medication interaction checking
        interactions = []
        for i, med1 in enumerate(medications):
            for med2 in medications[i+1:]:
                # Simplified interaction check
                if self._check_medication_interaction(med1, med2):
                    interactions.append({
                        'medications': [med1, med2],
                        'severity': random.choice(['Minor', 'Moderate', 'Major']),
                        'description': f"Potential interaction between {med1} and {med2}"
                    })

        # Check new medication if provided
        new_med_interactions = []
        if new_medication:
            for existing_med in medications:
                if self._check_medication_interaction(new_medication, existing_med):
                    new_med_interactions.append({
                        'medications': [new_medication, existing_med],
                        'severity': random.choice(['Minor', 'Moderate', 'Major']),
                        'description': f"Potential interaction between {new_medication} and {existing_med}"
                    })

        consultation_result = {
            'medications_reviewed': medications,
            'interaction_analysis': {
                'existing_interactions': interactions,
                'new_medication_interactions': new_med_interactions,
                'total_interactions_found': len(interactions) + len(new_med_interactions)
            },
            'safety_recommendations': self._generate_medication_safety_recommendations(
                interactions, new_med_interactions
            ),
            'adherence_tips': self._provide_adherence_tips(medications),
            'monitoring_requirements': self._suggest_monitoring_requirements(medications, patient_conditions)
        }

        return {
            'success': True,
            'operation_type': 'medication_consultation',
            'consultation_result': consultation_result,
            'pharmacist_recommendation': "Consult with pharmacist or physician about any concerns",
            'disclaimer': "This is educational information only. Always follow healthcare provider instructions."
        }

    def _wellness_planning(self, wellness_goals: Dict, **kwargs) -> Dict[str, Any]:
        """Create personalized wellness and prevention plan"""

        age = wellness_goals.get('age', 35)
        gender = wellness_goals.get('gender', 'unknown')
        health_goals = wellness_goals.get('goals', ['general_wellness'])
        current_conditions = wellness_goals.get('conditions', [])
        lifestyle_factors = wellness_goals.get('lifestyle', {})

        # Generate age-appropriate screening recommendations
        screening_recommendations = self._get_screening_recommendations(age, gender)

        # Create lifestyle recommendations
        lifestyle_plan = {
            'nutrition': self._create_nutrition_plan(health_goals, current_conditions),
            'exercise': self._create_exercise_plan(age, health_goals, current_conditions),
            'sleep': self._create_sleep_recommendations(),
            'stress_management': self._create_stress_management_plan(),
            'preventive_care': screening_recommendations
        }

        # Set measurable goals
        wellness_metrics = {
            'short_term_goals': self._set_short_term_goals(health_goals),
            'long_term_goals': self._set_long_term_goals(health_goals),
            'tracking_methods': self._recommend_tracking_methods(),
            'milestone_schedule': self._create_milestone_schedule()
        }

        return {
            'success': True,
            'operation_type': 'wellness_planning',
            'wellness_plan': lifestyle_plan,
            'wellness_metrics': wellness_metrics,
            'review_schedule': "Quarterly wellness plan review recommended",
            'healthcare_provider_note': "Share this plan with your healthcare provider for personalized guidance"
        }

    def _health_education(self, education_request: Dict, **kwargs) -> Dict[str, Any]:
        """Provide comprehensive health education on requested topics"""

        topics = education_request.get('topics', [])
        audience_level = education_request.get('level', 'general')  # basic, general, advanced
        format_preference = education_request.get('format', 'comprehensive')  # summary, comprehensive, detailed

        educational_content = {}

        for topic in topics:
            # Get health information for each topic
            info_result = self.execute_capability("health_information", topic, audience_level)

            if info_result.get('success'):
                topic_content = {
                    'basic_information': info_result['result'],
                    'practical_tips': self._generate_practical_tips(topic),
                    'warning_signs': self._identify_warning_signs(topic),
                    'when_to_seek_help': self._when_to_seek_help(topic),
                    'reliable_resources': self._recommend_reliable_resources(topic)
                }
                educational_content[topic] = topic_content

        # Create learning plan
        learning_plan = {
            'topics_covered': list(educational_content.keys()),
            'key_takeaways': self._extract_key_takeaways(educational_content),
            'action_items': self._create_educational_action_items(topics),
            'additional_learning': self._suggest_additional_learning(topics)
        }

        return {
            'success': True,
            'operation_type': 'health_education',
            'educational_content': educational_content,
            'learning_plan': learning_plan,
            'credibility_note': "Information based on general medical knowledge - consult healthcare providers for personalized advice"
        }

    # Helper methods for healthcare operations
    def _generate_possible_conditions(self, symptoms: List[str]) -> List[Dict[str, Any]]:
        """Generate possible conditions based on symptoms (educational only)"""

        # Simplified symptom-to-condition mapping
        condition_mapping = {
            'fever': ['Viral infection', 'Bacterial infection', 'Inflammatory condition'],
            'headache': ['Tension headache', 'Migraine', 'Sinus infection'],
            'cough': ['Common cold', 'Allergies', 'Respiratory infection'],
            'fatigue': ['Viral infection', 'Sleep deprivation', 'Stress'],
            'nausea': ['Gastroenteritis', 'Food poisoning', 'Medication side effect']
        }

        possible_conditions = []
        for symptom in symptoms:
            for keyword, conditions in condition_mapping.items():
                if keyword in symptom.lower():
                    for condition in conditions:
                        if condition not in [pc['condition'] for pc in possible_conditions]:
                            possible_conditions.append({
                                'condition': condition,
                                'likelihood': 'Possible',
                                'note': 'Educational information only - professional diagnosis required'
                            })

        return possible_conditions[:5]  # Limit to top 5

    def _generate_health_recommendations(self, symptoms: List[str], severity: str) -> List[str]:
        """Generate health recommendations based on symptoms and severity"""

        recommendations = []

        if severity == "EMERGENCY":
            recommendations = [
                "Call 911 or go to emergency room immediately",
                "Do not delay seeking emergency medical care",
                "Follow emergency dispatcher instructions if calling 911"
            ]
        elif severity == "URGENT":
            recommendations = [
                "Seek medical attention within 24 hours",
                "Contact healthcare provider immediately",
                "Go to urgent care if primary care unavailable",
                "Monitor symptoms closely for any worsening"
            ]
        else:
            recommendations = [
                "Monitor symptoms and their progression",
                "Rest and maintain adequate hydration",
                "Consider over-the-counter symptom relief if appropriate",
                "Schedule appointment with healthcare provider if symptoms persist"
            ]

        return recommendations

    def _get_care_urgency_guidance(self, severity: str) -> Dict[str, str]:
        """Get guidance on when to seek care based on severity"""

        urgency_guidance = {
            'EMERGENCY': {
                'timeframe': 'Immediately',
                'location': 'Emergency Room or call 911',
                'reason': 'Life-threatening condition possible'
            },
            'URGENT': {
                'timeframe': 'Within 24 hours',
                'location': 'Healthcare provider or urgent care',
                'reason': 'Prompt medical evaluation needed'
            },
            'ROUTINE': {
                'timeframe': 'Within 1-3 days if symptoms persist',
                'location': 'Primary care provider',
                'reason': 'Medical evaluation for persistent symptoms'
            }
        }

        return urgency_guidance.get(severity, urgency_guidance['ROUTINE'])

    def _get_self_care_options(self, symptoms: List[str]) -> List[str]:
        """Get appropriate self-care options for symptoms"""

        self_care_options = []

        for symptom in symptoms:
            symptom_lower = symptom.lower()

            if 'fever' in symptom_lower:
                self_care_options.extend(['Rest', 'Increase fluid intake', 'Consider fever reducer if appropriate'])
            elif 'cough' in symptom_lower:
                self_care_options.extend(['Honey for cough relief', 'Humidifier use', 'Throat lozenges'])
            elif 'headache' in symptom_lower:
                self_care_options.extend(['Rest in quiet, dark room', 'Apply cold or warm compress', 'Stay hydrated'])
            elif 'nausea' in symptom_lower:
                self_care_options.extend(['Clear fluids', 'BRAT diet', 'Ginger tea'])

        # Remove duplicates and add general options
        self_care_options = list(set(self_care_options))
        self_care_options.extend(['Adequate rest', 'Stress reduction', 'Good hygiene practices'])

        return self_care_options[:8]  # Limit to 8 options

    def _identify_red_flags(self, symptoms: List[str]) -> List[str]:
        """Identify red flag symptoms that require immediate attention"""

        red_flags = []
        emergency_indicators = {
            'chest pain': 'Severe chest pain, especially with shortness of breath',
            'difficulty breathing': 'Severe breathing difficulty or shortness of breath',
            'unconscious': 'Loss of consciousness or severe confusion',
            'severe bleeding': 'Uncontrolled bleeding',
            'severe allergic': 'Signs of severe allergic reaction (difficulty breathing, swelling)',
            'stroke': 'Signs of stroke (face drooping, arm weakness, speech difficulty)',
            'severe headache': 'Sudden, severe headache unlike any before'
        }

        for symptom in symptoms:
            symptom_lower = symptom.lower()
            for indicator, description in emergency_indicators.items():
                if indicator in symptom_lower:
                    red_flags.append(description)

        return red_flags

    def _get_monitoring_instructions(self, symptoms: List[str]) -> List[str]:
        """Get specific monitoring instructions for symptoms"""

        monitoring_instructions = [
            "Monitor temperature regularly",
            "Track symptom progression and severity",
            "Note any new symptoms that develop",
            "Keep a symptom diary with times and descriptions"
        ]

        # Add symptom-specific monitoring
        for symptom in symptoms:
            if 'pain' in symptom.lower():
                monitoring_instructions.append("Rate pain level on 1-10 scale regularly")
            elif 'fever' in symptom.lower():
                monitoring_instructions.append("Check temperature every 4-6 hours")

        return monitoring_instructions

    def _get_escalation_criteria(self) -> List[str]:
        """Get criteria for when to escalate care level"""

        return [
            "Symptoms worsen significantly",
            "New concerning symptoms develop",
            "High fever (>103°F) develops",
            "Difficulty breathing occurs",
            "Severe pain develops",
            "Signs of dehydration appear",
            "Confusion or altered mental state occurs"
        ]

    def _check_medication_interaction(self, med1: str, med2: str) -> bool:
        """Simple medication interaction check (simplified for demo)"""
        # In reality, this would check against a comprehensive drug interaction database

        # Simulate some known interactions
        known_interactions = [
            ('warfarin', 'aspirin'),
            ('metformin', 'alcohol'),
            ('lisinopril', 'potassium'),
            ('simvastatin', 'grapefruit')
        ]

        med1_lower = med1.lower()
        med2_lower = med2.lower()

        for interaction in known_interactions:
            if (interaction[0] in med1_lower and interaction[1] in med2_lower) or \
               (interaction[1] in med1_lower and interaction[0] in med2_lower):
                return True

        # Random chance for demo purposes
        return random.random() < 0.2

    def _generate_medication_safety_recommendations(self, existing_interactions: List, new_interactions: List) -> List[str]:
        """Generate medication safety recommendations based on interactions"""

        recommendations = []

        total_interactions = len(existing_interactions) + len(new_interactions)

        if total_interactions == 0:
            recommendations.append("No obvious interactions identified in current review")
        else:
            recommendations.append(f"Found {total_interactions} potential interaction(s) requiring attention")

        if any(interaction['severity'] == 'Major' for interaction in existing_interactions + new_interactions):
            recommendations.append("IMPORTANT: Major interaction identified - consult healthcare provider immediately")

        recommendations.extend([
            "Always inform healthcare providers of all medications you take",
            "Include over-the-counter medications and supplements",
            "Use the same pharmacy when possible for interaction checking",
            "Never stop medications without consulting healthcare provider"
        ])

        return recommendations

    def _provide_adherence_tips(self, medications: List[str]) -> List[str]:
        """Provide medication adherence tips"""

        return [
            "Take medications at the same time each day",
            "Use pill organizers to track daily doses",
            "Set phone alarms as reminders",
            "Keep medications in visible locations",
            "Never skip doses without consulting healthcare provider",
            "Refill prescriptions before running out",
            "Understand what each medication is for",
            "Report side effects to healthcare provider"
        ]

    def _suggest_monitoring_requirements(self, medications: List[str], conditions: List[str]) -> List[str]:
        """Suggest monitoring requirements for medications and conditions"""

        monitoring_suggestions = []

        # General monitoring
        monitoring_suggestions.extend([
            "Regular blood pressure checks if taking BP medications",
            "Blood sugar monitoring for diabetes medications",
            "Liver function tests for certain medications",
            "Kidney function monitoring for some drugs"
        ])

        # Condition-specific monitoring
        for condition in conditions:
            if 'diabetes' in condition.lower():
                monitoring_suggestions.append("Regular A1C and blood glucose monitoring")
            elif 'hypertension' in condition.lower():
                monitoring_suggestions.append("Daily blood pressure monitoring")
            elif 'heart' in condition.lower():
                monitoring_suggestions.append("Regular cardiac function assessments")

        return monitoring_suggestions[:6]  # Limit to 6 most relevant

    def _generate_health_action_plan(self, symptom_analysis: Dict, triage_assessment: Dict) -> List[str]:
        """Generate comprehensive health action plan"""

        action_plan = []

        # Add triage-based actions
        priority_level = triage_assessment.get('priority_level', 'ROUTINE')
        if 'CRITICAL' in priority_level or 'EMERGENCY' in priority_level:
            action_plan.extend(triage_assessment.get('immediate_actions', []))
        elif 'URGENT' in priority_level:
            action_plan.extend([
                "Seek medical attention within 24 hours",
                "Monitor symptoms closely"
            ])
        else:
            action_plan.extend([
                "Continue monitoring symptoms",
                "Use appropriate self-care measures"
            ])

        # Add symptom-specific actions
        action_plan.extend(symptom_analysis.get('recommendations', []))

        return action_plan

    def _create_follow_up_plan(self, symptom_analysis: Dict) -> Dict[str, Any]:
        """Create follow-up plan based on symptom analysis"""

        severity = symptom_analysis.get('severity_level', 'ROUTINE')

        if severity == 'EMERGENCY':
            timeframe = 'Follow emergency care instructions'
        elif severity == 'URGENT':
            timeframe = '24-48 hours for medical follow-up'
        else:
            timeframe = '1-2 weeks if symptoms persist'

        return {
            'follow_up_timeframe': timeframe,
            'monitoring_period': '72 hours for symptom progression',
            'reassessment_triggers': [
                'Symptoms worsen significantly',
                'New symptoms develop',
                'No improvement after expected timeframe'
            ],
            'next_steps': [
                'Document symptom progression',
                'Prepare questions for healthcare provider',
                'Gather relevant medical history'
            ]
        }

    def _recommend_educational_resources(self, symptoms: List[str]) -> List[str]:
        """Recommend educational resources based on symptoms"""

        resources = [
            "Mayo Clinic - Reliable medical information",
            "WebMD - Symptom checker and health information",
            "CDC - Disease prevention and health promotion",
            "National Institutes of Health (NIH) - Medical research"
        ]

        # Add symptom-specific resources
        for symptom in symptoms:
            if 'heart' in symptom.lower() or 'chest' in symptom.lower():
                resources.append("American Heart Association - Heart health information")
            elif 'diabetes' in symptom.lower() or 'blood sugar' in symptom.lower():
                resources.append("American Diabetes Association - Diabetes management")

        return resources[:5]  # Limit to 5 resources

# =============================================================================
# LEGAL ADVISOR AGENT
# =============================================================================

class LegalAdvisorAgent(CustomAgentBase):
    """
    Specialized agent for legal research, contract analysis,
    and legal information provision.

    This agent provides legal information while emphasizing
    the need for professional legal counsel.
    """

    def __init__(self, name: str = "LegalAdvisor", practice_area: str = "general"):
        super().__init__(name=name, domain=AgentDomain.LEGAL)

        self.practice_area = practice_area  # general, corporate, family, criminal, etc.
        self.setup_legal_capabilities()
        self.setup_legal_traits()
        self.setup_legal_knowledge()

        print(f"⚖️ Legal Advisor Agent '{name}' initialized")
        print(f"   Practice area: {practice_area}")
        print(f"   ⚠️  Legal disclaimer: This provides legal information only, not legal advice")

    def setup_legal_capabilities(self):
        """Setup legal analysis capabilities"""

        # Contract Analysis Capability
        def contract_analysis(contract_text: str, analysis_type: str = "general"):
            """Analyze contract for key terms, risks, and recommendations"""

            if not contract_text:
                return {'error': 'No contract text provided for analysis'}

            # Simulate contract analysis
            word_count = len(contract_text.split())

            # Identify key sections (simplified)
            key_sections = []
            section_keywords = {
                'parties': ['party', 'parties', 'between'],
                'consideration': ['payment', 'consideration', 'fee', 'price'],
                'termination': ['terminate', 'termination', 'end', 'expiry'],
                'liability': ['liable', 'liability', 'responsible', 'damages'],
                'dispute_resolution': ['dispute', 'arbitration', 'court', 'litigation']
            }

            for section, keywords in section_keywords.items():
                if any(keyword in contract_text.lower() for keyword in keywords):
                    key_sections.append(section)

            # Identify potential risks
            risk_indicators = [
                'unlimited liability', 'no warranty', 'automatic renewal',
                'penalty', 'liquidated damages', 'non-compete'
            ]

            identified_risks = [
                risk for risk in risk_indicators
                if risk in contract_text.lower()
            ]

            # Generate recommendations
            recommendations = self._generate_contract_recommendations(
                key_sections, identified_risks, word_count
            )

            return {
                'contract_summary': {
                    'word_count': word_count,
                    'key_sections_identified': key_sections,
                    'completeness_score': len(key_sections) / len(section_keywords)
                },
                'risk_analysis': {
                    'identified_risks': identified_risks,
                    'risk_level': 'HIGH' if len(identified_risks) > 3 else 'MODERATE' if len(identified_risks) > 1 else 'LOW'
                },
                'recommendations': recommendations,
                'missing_sections': [
                    section for section in section_keywords.keys()
                    if section not in key_sections
                ],
                'legal_disclaimer': "This is informational analysis only. Consult a qualified attorney for legal advice."
            }

        contract_capability = AgentCapability(
            capability_id="contract_analysis",
            name="Contract Analysis",
            capability_type=CapabilityType.ANALYTICAL,
            expertise_level=ExpertiseLevel.ADVANCED,
            description="Legal contract analysis and risk assessment",
            accuracy_rating=0.85,
            speed_rating=0.80,
            reliability_rating=0.90,
            execution_function=contract_analysis
        )
        self.add_capability(contract_capability)

        # Legal Research Capability
        def legal_research(topic: str, jurisdiction: str = "general", research_depth: str = "standard"):
            """Conduct legal research on specified topics"""

            # Simulate legal research database
            legal_topics = {
                'contract_law': {
                    'key_principles': ['Offer', 'Acceptance', 'Consideration', 'Capacity', 'Legality'],
                    'recent_developments': ['Electronic signature validity', 'Remote contract formation'],
                    'relevant_statutes': ['Uniform Commercial Code', 'Statute of Frauds'],
                    'case_law_trends': ['Increased enforcement of arbitration clauses']
                },
                'employment_law': {
                    'key_principles': ['At-will employment', 'Discrimination protections', 'Wage and hour laws'],
                    'recent_developments': ['Remote work regulations', 'AI in hiring decisions'],
                    'relevant_statutes': ['Title VII', 'ADA', 'FLSA'],
                    'case_law_trends': ['Expansion of harassment definitions']
                },
                'intellectual_property': {
                    'key_principles': ['Copyright', 'Trademark', 'Patent', 'Trade secrets'],
                    'recent_developments': ['AI-generated content ownership', 'Software patentability'],
                    'relevant_statutes': ['Copyright Act', 'Lanham Act', 'Patent Act'],
                    'case_law_trends': ['Fair use in digital contexts']
                }
            }

            topic_info = legal_topics.get(topic.lower().replace(' ', '_'))
            if not topic_info:
                return {
                    'error': f'Legal topic "{topic}" not found in research database',
                    'available_topics': list(legal_topics.keys()),
                    'research_suggestion': 'Consult legal databases or attorney for comprehensive research'
                }

            research_result = {
                'topic': topic,
                'jurisdiction': jurisdiction,
                'research_depth': research_depth,
                'key_legal_principles': topic_info['key_principles'],
                'recent_legal_developments': topic_info['recent_developments'],
                'relevant_statutes': topic_info['relevant_statutes'],
                'case_law_trends': topic_info['case_law_trends'],
                'research_limitations': [
                    'Limited to general legal principles',
                    'Not jurisdiction-specific unless specified',
                    'May not include most recent developments'
                ],
                'next_steps': [
                    'Consult jurisdiction-specific legal databases',
                    'Review recent case law in relevant courts',
                    'Consider consulting with specialized attorney'
                ]
            }

            return research_result

        research_capability = AgentCapability(
            capability_id="legal_research",
            name="Legal Research",
            capability_type=CapabilityType.ANALYTICAL,
            expertise_level=ExpertiseLevel.ADVANCED,
            description="Legal research and case law analysis",
            accuracy_rating=0.80,
            speed_rating=0.85,
            reliability_rating=0.88,
            execution_function=legal_research
        )
        self.add_capability(research_capability)

        # Document Drafting Capability
        def document_drafting(document_type: str, parameters: Dict):
            """Draft legal documents based on provided parameters"""

            # Document templates (simplified)
            document_templates = {
                'nda': {
                    'required_fields': ['parties', 'confidential_information_definition', 'duration'],
                    'optional_fields': ['governing_law', 'dispute_resolution'],
                    'template_sections': [
                        'Parties identification',
                        'Definition of confidential information',
                        'Obligations of receiving party',
                        'Exceptions to confidentiality',
                        'Term and termination',
                        'Remedies for breach',
                        'Governing law'
                    ]
                },
                'service_agreement': {
                    'required_fields': ['parties', 'services_description', 'payment_terms'],
                    'optional_fields': ['liability_limitations', 'intellectual_property'],
                    'template_sections': [
                        'Service description',
                        'Payment terms and schedule',
                        'Performance standards',
                        'Intellectual property ownership',
                        'Liability and indemnification',
                        'Termination provisions'
                    ]
                }
            }

            template_info = document_templates.get(document_type.lower())
            if not template_info:
                return {
                    'error': f'Document type "{document_type}" not available',
                    'available_types': list(document_templates.keys())
                }

            # Check required fields
            missing_fields = [
                field for field in template_info['required_fields']
                if field not in parameters
            ]

            if missing_fields:
                return {
                    'error': 'Missing required fields',
                    'missing_fields': missing_fields,
                    'required_fields': template_info['required_fields']
                }

            # Generate document outline
            document_outline = {
                'document_type': document_type,
                'sections': template_info['template_sections'],
                'provided_parameters': parameters,
                'customization_notes': self._generate_customization_notes(document_type, parameters),
                'review_checklist': self._generate_review_checklist(document_type),
                'attorney_review_recommended': True,
                'disclaimer': 'This is a template outline only. Professional legal review is essential.'
            }

            return document_outline

        drafting_capability = AgentCapability(
            capability_id="document_drafting",
            name="Legal Document Drafting",
            capability_type=CapabilityType.CREATIVE,
            expertise_level=ExpertiseLevel.INTERMEDIATE,
            description="Legal document template creation and customization",
            accuracy_rating=0.75,
            speed_rating=0.90,
            reliability_rating=0.85,
            execution_function=document_drafting
        )
        self.add_capability(drafting_capability)

    def setup_legal_traits(self):
        """Setup legal-specific behavioral traits"""

        # Precision-Focused Trait
        precision_trait = AgentTrait(
            trait_id="precision_focused",
            name="Precision-Focused Analysis",
            description="Emphasis on precise language and careful analysis",
            intensity=0.9,
            decision_modifier=0.25,  # Very careful decisions
            speed_modifier=-0.2,    # Slower but more thorough
            communication_modifier=0.15,  # Precise communication
            activation_contexts=["legal_analysis", "contract_review", "document_drafting"]
        )
        self.add_trait(precision_trait)

        # Conservative Advisory Trait
        conservative_advisory_trait = AgentTrait(
            trait_id="conservative_advisory",
            name="Conservative Legal Advisory",
            description="Conservative approach emphasizing legal compliance and risk management",
            intensity=0.85,
            decision_modifier=0.2,
            risk_modifier=-0.3,   # Risk-averse
            activation_contexts=["legal_advice", "compliance_review", "risk_assessment"]
        )
        self.add_trait(conservative_advisory_trait)

        # Professional Disclaimer Trait
        disclaimer_trait = AgentTrait(
            trait_id="professional_disclaimer",
            name="Professional Disclaimer Emphasis",
            description="Consistent emphasis on the need for professional legal counsel",
            intensity=1.0,
            communication_modifier=0.3,  # Clear about limitations
            activation_contexts=["legal_information", "legal_analysis", "document_review"]
        )
        self.add_trait(disclaimer_trait)

    def setup_legal_knowledge(self):
        """Setup comprehensive legal knowledge base"""

        # Add legal facts
        self.domain_knowledge.add_fact(
            "contract_formation",
            "A valid contract requires offer, acceptance, consideration, capacity, and legality",
            confidence=1.0,
            tags=["contract_law", "formation", "elements"]
        )

        self.domain_knowledge.add_fact(
            "statute_of_limitations",
            "Statute of limitations sets time limits for bringing legal claims",
            confidence=1.0,
            tags=["procedure", "time_limits", "claims"]
        )

        self.domain_knowledge.add_fact(
            "attorney_client_privilege",
            "Communications between attorney and client are generally privileged and confidential",
            confidence=0.95,
            tags=["privilege", "confidentiality", "ethics"]
        )

        # Add legal rules
        self.domain_knowledge.add_rule(
            "contract_dispute_rule",
            "If contract terms are ambiguous",
            "Interpret against the party who drafted the contract",
            confidence=0.85
        )

        self.domain_knowledge.add_rule(
            "employment_termination_rule",
            "If employee termination involves discrimination claims",
            "Document all performance issues and follow progressive discipline",
            confidence=0.90
        )

        # Add legal procedures
        self.domain_knowledge.add_procedure(
            "contract_negotiation",
            "Contract Negotiation Process",
            [
                "Identify key terms and objectives",
                "Research market standards",
                "Prepare initial draft or review counterpart draft",
                "Negotiate terms systematically",
                "Document all changes and agreements",
                "Obtain necessary approvals",
                "Execute final agreement"
            ],
            prerequisites=["clear_objectives", "authority_to_negotiate"]
        )

        self.domain_knowledge.add_procedure(
            "legal_compliance_review",
            "Legal Compliance Review Process",
            [
                "Identify applicable laws and regulations",
                "Review current practices and policies",
                "Identify compliance gaps",
                "Develop remediation plan",
                "Implement compliance measures",
                "Monitor and audit compliance",
                "Update as laws change"
            ]
        )

        # Add legal terminology
        legal_terms = {
            "Force Majeure": "Unforeseeable circumstances that prevent contract performance",
            "Indemnification": "Agreement to compensate for harm or loss",
            "Liquidated Damages": "Pre-agreed amount of damages for contract breach",
            "Jurisdiction": "Court's authority to hear and decide cases",
            "Due Diligence": "Investigation or exercise of care that reasonable person would take",
            "Breach of Contract": "Failure to perform any term of a contract without legitimate legal excuse",
            "Intellectual Property": "Creations of the mind protected by law",
            "Confidentiality": "Duty to keep information secret"
        }

        for term, definition in legal_terms.items():
            self.domain_knowledge.add_terminology(term, definition)

    def specialized_operation(self, operation_type: str, **kwargs) -> Dict[str, Any]:
        """Legal advisor specialized operations"""

        if operation_type == "comprehensive_contract_review":
            return self._comprehensive_contract_review(**kwargs)

        elif operation_type == "legal_compliance_assessment":
            return self._legal_compliance_assessment(**kwargs)

        elif operation_type == "legal_strategy_development":
            return self._legal_strategy_development(**kwargs)

        elif operation_type == "risk_mitigation_planning":
            return self._risk_mitigation_planning(**kwargs)

        else:
            return {
                'success': False,
                'error': f'Unknown legal operation: {operation_type}',
                'available_operations': [
                    'comprehensive_contract_review',
                    'legal_compliance_assessment',
                    'legal_strategy_development',
                    'risk_mitigation_planning'
                ]
            }

    def _comprehensive_contract_review(self, contract_data: Dict, **kwargs) -> Dict[str, Any]:
        """Comprehensive contract review combining multiple legal capabilities"""

        contract_text = contract_data.get('contract_text', '')
        contract_type = contract_data.get('type', 'general')

        # Perform contract analysis
        analysis_result = self.execute_capability("contract_analysis", contract_text)

        # Conduct relevant legal research
        research_result = self.execute_capability("legal_research", "contract_law")

        # Generate comprehensive review
        contract_review = {
            'review_date': datetime.now().isoformat(),
            'contract_analysis': analysis_result.get('result', {}),
            'legal_research_context': research_result.get('result', {}),
            'detailed_recommendations': self._generate_detailed_contract_recommendations(
                analysis_result.get('result', {}),
                contract_type
            ),
            'risk_mitigation_strategies': self._develop_contract_risk_strategies(
                analysis_result.get('result', {})
            ),
            'negotiation_priorities': self._identify_negotiation_priorities(
                analysis_result.get('result', {})
            ),
            'attorney_consultation_areas': self._identify_attorney_consultation_needs(
                analysis_result.get('result', {})
            )
        }

        return {
            'success': True,
            'operation_type': 'comprehensive_contract_review',
            'contract_review': contract_review,
            'professional_recommendation': "Have this contract reviewed by qualified legal counsel before execution",
            'legal_disclaimer': "This review is for informational purposes only and does not constitute legal advice"
        }

    # Helper methods for legal operations
    def _generate_detailed_contract_recommendations(self, analysis_result: Dict, contract_type: str) -> List[str]:
        """Generate detailed contract recommendations based on analysis"""

        recommendations = []

        # Basic recommendations from analysis
        basic_recs = analysis_result.get('recommendations', [])
        recommendations.extend(basic_recs)

        # Contract type specific recommendations
        if contract_type == 'service_agreement':
            recommendations.extend([
                "Define clear scope of work and deliverables",
                "Include specific performance metrics and acceptance criteria",
                "Add provisions for scope changes and additional work"
            ])
        elif contract_type == 'nda':
            recommendations.extend([
                "Clearly define what constitutes confidential information",
                "Set reasonable duration for confidentiality obligations",
                "Include standard exceptions (publicly available information, etc.)"
            ])

        # Risk-based recommendations
        risk_level = analysis_result.get('risk_analysis', {}).get('risk_level', 'MODERATE')
        if risk_level == 'HIGH':
            recommendations.extend([
                "URGENT: Address high-risk provisions before execution",
                "Consider additional protective clauses",
                "Seek specialized legal counsel for risk mitigation"
            ])

        # Missing section recommendations
        missing_sections = analysis_result.get('missing_sections', [])
        for section in missing_sections:
            recommendations.append(f"Add {section.replace('_', ' ')} section to contract")

        return recommendations

    def _develop_contract_risk_strategies(self, analysis_result: Dict) -> List[str]:
        """Develop risk mitigation strategies for contract"""

        strategies = []
        identified_risks = analysis_result.get('risk_analysis', {}).get('identified_risks', [])

        for risk in identified_risks:
            if 'unlimited liability' in risk:
                strategies.append("Negotiate liability caps and limitations")
            elif 'automatic renewal' in risk:
                strategies.append("Add termination notice provisions")
            elif 'penalty' in risk:
                strategies.append("Review penalty clauses for reasonableness")

        # General risk mitigation strategies
        strategies.extend([
            "Include force majeure provisions",
            "Add dispute resolution mechanisms",
            "Ensure compliance with applicable laws"
        ])

        return strategies

    def _identify_negotiation_priorities(self, analysis_result: Dict) -> List[str]:
        """Identify key negotiation priorities"""

        priorities = []
        risk_level = analysis_result.get('risk_analysis', {}).get('risk_level', 'MODERATE')

        if risk_level == 'HIGH':
            priorities.append("Priority 1: Address high-risk provisions")

        missing_sections = analysis_result.get('missing_sections', [])
        if 'liability' in missing_sections:
            priorities.append("Priority 2: Add liability and indemnification terms")

        if 'termination' in missing_sections:
            priorities.append("Priority 3: Include termination provisions")

        # Standard negotiation priorities
        priorities.extend([
            "Clarify payment terms and schedules",
            "Define performance standards and remedies",
            "Establish intellectual property ownership"
        ])

        return priorities

    def _identify_attorney_consultation_needs(self, analysis_result: Dict) -> List[str]:
        """Identify areas requiring attorney consultation"""

        consultation_needs = []

        # Always recommend attorney review
        consultation_needs.append("Professional legal review recommended for all contracts")

        # Specific consultation needs based on analysis
        risk_level = analysis_result.get('risk_analysis', {}).get('risk_level', 'MODERATE')
        if risk_level == 'HIGH':
            consultation_needs.append("URGENT: High-risk contract requires immediate legal counsel")

        identified_risks = analysis_result.get('risk_analysis', {}).get('identified_risks', [])
        if len(identified_risks) > 3:
            consultation_needs.append("Multiple risk factors require professional legal analysis")

        missing_sections = analysis_result.get('missing_sections', [])
        if len(missing_sections) > 2:
            consultation_needs.append("Significant missing provisions require legal drafting expertise")

        return consultation_needs
    def _generate_contract_recommendations(self, key_sections: List[str],
                                         identified_risks: List[str], word_count: int) -> List[str]:
        """Generate contract recommendations based on analysis"""

        recommendations = []

        # Length-based recommendations
        if word_count < 500:
            recommendations.append("Contract appears brief - consider if all necessary terms are included")
        elif word_count > 5000:
            recommendations.append("Contract is lengthy - ensure all terms are clearly understood")

        # Section-based recommendations
        if 'liability' not in key_sections:
            recommendations.append("Consider adding liability and indemnification provisions")

        if 'dispute_resolution' not in key_sections:
            recommendations.append("Add dispute resolution mechanism (arbitration or litigation)")

        if 'termination' not in key_sections:
            recommendations.append("Include clear termination provisions and procedures")

        # Risk-based recommendations
        if identified_risks:
            recommendations.append(f"Address {len(identified_risks)} potential risk factors identified")

        if 'unlimited liability' in identified_risks:
            recommendations.append("CRITICAL: Review unlimited liability provisions carefully")

        return recommendations

    def _generate_customization_notes(self, document_type: str, parameters: Dict) -> List[str]:
        """Generate customization notes for document drafting"""

        notes = [
            "Customize all bracketed placeholders with specific information",
            "Review all provisions for applicability to your situation",
            "Consider jurisdiction-specific requirements"
        ]

        if document_type == 'nda':
            notes.extend([
                "Define confidential information precisely",
                "Set appropriate duration for confidentiality obligations",
                "Consider carve-outs for publicly available information"
            ])
        elif document_type == 'service_agreement':
            notes.extend([
                "Specify detailed scope of services",
                "Include clear payment terms and schedule",
                "Define performance standards and metrics"
            ])

        return notes

    def _generate_review_checklist(self, document_type: str) -> List[str]:
        """Generate review checklist for drafted documents"""

        checklist = [
            "All parties correctly identified",
            "All terms clearly defined",
            "Legal compliance verified",
            "Signatures and dates complete"
        ]

        if document_type == 'nda':
            checklist.extend([
                "Confidential information scope appropriate",
                "Duration reasonable and enforceable",
                "Remedies for breach adequate"
            ])
        elif document_type == 'service_agreement':
            checklist.extend([
                "Scope of work clearly defined",
                "Payment terms and schedule specified",
                "Performance standards established"
            ])

        return checklist

# =============================================================================
# TEST THE SPECIALIZED AGENTS
# =============================================================================

print("\n🧪 Testing Specialized Agent Types...")
print()

# Test Financial Analyst Agent
print("💰 Testing Financial Analyst Agent:")
financial_agent = FinancialAnalystAgent("WealthAdvisor", risk_tolerance="moderate")

# Test portfolio analysis
sample_portfolio = {
    'holdings': [
        {'symbol': 'AAPL', 'value': 50000, 'beta': 1.2},
        {'symbol': 'GOOGL', 'value': 30000, 'beta': 1.1},
        {'symbol': 'BND', 'value': 20000, 'beta': 0.1}
    ],
    'market_data': {'market_direction': 'neutral'}
}

portfolio_analysis = financial_agent.specialized_operation(
    "complete_portfolio_analysis",
    portfolio_data=sample_portfolio
)

if portfolio_analysis['success']:
    analysis_report = portfolio_analysis.get('analysis_report', {})
    summary = analysis_report.get('executive_summary', 'Analysis completed successfully')
    print(f"   Portfolio analysis completed:")
    print(f"   {summary.split(chr(10))[0] if chr(10) in summary else summary}")  # First line of summary
    recommendations = analysis_report.get('recommendations', [])
    print(f"   Recommendations: {len(recommendations)}")
else:
    print(f"   Portfolio analysis failed: {portfolio_analysis.get('error', 'Unknown error')}")

# Test Healthcare Advisor Agent
print("\n🏥 Testing Healthcare Advisor Agent:")
health_agent = HealthcareAdvisorAgent("HealthGuide", specialty="general")

# Test symptom analysis
test_symptoms = ["mild headache", "fatigue", "low fever"]
health_assessment = health_agent.specialized_operation(
    "comprehensive_health_assessment",
    health_data={
        'symptoms': test_symptoms,
        'patient_info': {'age': 30, 'gender': 'adult'}
    }
)

if health_assessment['success']:
    assessment_data = health_assessment.get('health_assessment', {})
    symptom_analysis = assessment_data.get('symptom_analysis', {})
    severity = symptom_analysis.get('severity_level', 'Unknown')
    recommendations = symptom_analysis.get('recommendations', [])
    recommendations_count = len(recommendations)
    print(f"   Health assessment completed:")
    print(f"   Severity level: {severity}")
    print(f"   Recommendations provided: {recommendations_count}")
else:
    print(f"   Health assessment failed: {health_assessment.get('error', 'Unknown error')}")

# Test Legal Advisor Agent
print("\n⚖️ Testing Legal Advisor Agent:")
legal_agent = LegalAdvisorAgent("LegalGuide", practice_area="contract")

# Test contract analysis
sample_contract = """
This Service Agreement is between Company A and Company B.
Company B will provide consulting services for a fee of $10,000.
The agreement terminates after 6 months.
Payment is due within 30 days of invoice.
Company B is liable for any damages caused by negligence.
"""

contract_review = legal_agent.specialized_operation(
    "comprehensive_contract_review",
    contract_data={
        'contract_text': sample_contract,
        'type': 'service_agreement'
    }
)

if contract_review['success']:
    review_data = contract_review.get('contract_review', {})
    analysis = review_data.get('contract_analysis', {})
    risk_analysis = analysis.get('risk_analysis', {})
    risk_level = risk_analysis.get('risk_level', 'Unknown')
    recommendations = analysis.get('recommendations', [])
    recommendations_count = len(recommendations)
    print(f"   Contract review completed:")
    print(f"   Risk level: {risk_level}")
    print(f"   Recommendations provided: {recommendations_count}")
else:
    print(f"   Contract review failed: {contract_review.get('error', 'Unknown error')}")

print()
print("🎉 All Specialized Agent Types working correctly!")
print()
print("📋 Summary of Part 2:")
print("   • FinancialAnalystAgent - Portfolio analysis, market prediction, risk assessment")
print("   • HealthcareAdvisorAgent - Symptom analysis, health information, emergency triage")
print("   • LegalAdvisorAgent - Contract analysis, legal research, document drafting")
print("   • Domain-specific knowledge bases with professional disclaimers")
print("   • Specialized capabilities with appropriate expertise levels")
print("   • Professional behavioral traits for each domain")
print("   • Comprehensive operation methods for complex workflows")
print()
print("   Ready for Part 3: Dynamic Composition & Behavior Control")

"""
This completes Tutorial 14 Part 2: Specialized Agent Types

What we built:
- Three complete specialized agent implementations
- Domain-specific capabilities with realistic performance characteristics
- Professional behavioral traits appropriate for each field
- Comprehensive knowledge bases with facts, rules, procedures, and terminology
- Complex specialized operations that combine multiple capabilities
- Appropriate safety disclaimers and professional recommendations

Next: Tutorial 14 Part 3 will cover dynamic behavior control and agent composition.
"""


🧪 Testing Specialized Agent Types...

💰 Testing Financial Analyst Agent:
📚 Domain knowledge initialized for: financial
🤖 Custom agent created: WealthAdvisor
   Domain: financial
   Agent ID: fd8da1c9...
   ➕ Added capability: Portfolio Analysis (ADVANCED)
   ➕ Added capability: Market Prediction (ADVANCED)
   ➕ Added capability: Investment Risk Assessment (MASTER)
   🎭 Added trait: Conservative Analysis (intensity: 0.40)
   🎭 Added trait: Detail-Oriented Analysis (intensity: 0.90)
   🎭 Added trait: Market Awareness (intensity: 0.85)
   📝 Added fact: Compound interest is interest calculated on the in...
   📝 Added fact: Diversification reduces risk by spreading investme...
   📝 Added fact: Higher potential returns generally come with highe...
   📋 Added rule: If investor age > 30 and risk_tolerance = conservative → Allocate maximum 60% to equities, 40% to bonds and cash
   📋 Added rule: If emergency_fund < 6_months_expenses → Prioritize building emergency fund before aggressive investi

'\nThis completes Tutorial 14 Part 2: Specialized Agent Types\n\nWhat we built:\n- Three complete specialized agent implementations\n- Domain-specific capabilities with realistic performance characteristics\n- Professional behavioral traits appropriate for each field\n- Comprehensive knowledge bases with facts, rules, procedures, and terminology\n- Complex specialized operations that combine multiple capabilities\n- Appropriate safety disclaimers and professional recommendations\n\nNext: Tutorial 14 Part 3 will cover dynamic behavior control and agent composition.\n'

In [8]:
"""
Tutorial 14 Part 3: Dynamic Composition & Behavior Control
==========================================================

Advanced agent behavior management, team formation, and dynamic
personality adaptation. This part shows how to create intelligent
agent teams that adapt their behavior based on context.

Prerequisites: Run Tutorial 14 Parts 1 and 2 first.
"""

import random
import math
from datetime import datetime, timedelta
from typing import List, Dict, Any, Optional, Callable, Set
from collections import defaultdict
from dataclasses import dataclass, field
import json
import time

print("Tutorial 14 Part 3: Dynamic Composition & Behavior Control")
print("=" * 54)
print("Building intelligent agent teams and behavior adaptation...")
print()


Tutorial 14 Part 3: Dynamic Composition & Behavior Control
Building intelligent agent teams and behavior adaptation...



In [16]:
# BEHAVIOR CONTROLLER - DYNAMIC PERSONALITY MANAGEMENT
# =============================================================================

class BehaviorController:
    """
    Controls and adapts agent behavior dynamically based on context,
    performance, and environmental factors.

    This enables agents to modify their personality and approach
    in real-time for optimal performance.
    """

    def __init__(self, agent_instance):
        self.agent = agent_instance
        self.behavior_history = []
        self.performance_metrics = {}
        self.adaptation_rules = {}
        self.context_sensitivity = 0.7  # How much context affects behavior

        print(f"🎭 Behavior Controller initialized for {agent_instance.name}")
        self.setup_adaptation_rules()

    def setup_adaptation_rules(self):
        """Setup rules for dynamic behavior adaptation"""

        # Performance-based adaptations
        self.adaptation_rules['performance'] = {
            'low_success_rate': {
                'trigger': lambda metrics: metrics.get('success_rate', 1.0) < 0.6,
                'adaptations': {
                    'increase_caution': 0.2,
                    'decrease_speed': 0.1,
                    'increase_detail_focus': 0.15
                }
            },
            'high_success_rate': {
                'trigger': lambda metrics: metrics.get('success_rate', 0.0) > 0.9,
                'adaptations': {
                    'increase_confidence': 0.1,
                    'increase_speed': 0.05
                }
            }
        }

        # Context-based adaptations
        self.adaptation_rules['context'] = {
            'high_pressure': {
                'trigger': lambda context: context.get('urgency') == 'high' or context.get('stakes') == 'high',
                'adaptations': {
                    'increase_focus': 0.3,
                    'decrease_risk_tolerance': 0.2,
                    'increase_verification': 0.25
                }
            },
            'collaborative': {
                'trigger': lambda context: context.get('team_size', 0) > 1,
                'adaptations': {
                    'increase_communication': 0.2,
                    'increase_cooperation': 0.15,
                    'decrease_independence': 0.1
                }
            },
            'time_pressure': {
                'trigger': lambda context: context.get('deadline_pressure', 0) > 0.7,
                'adaptations': {
                    'increase_speed': 0.3,
                    'decrease_thoroughness': 0.15,
                    'focus_essentials': 0.2
                }
            }
        }

        # Domain-specific adaptations
        if hasattr(self.agent, 'domain'):
            domain_adaptations = self._get_domain_specific_adaptations()
            self.adaptation_rules['domain'] = domain_adaptations

    def adapt_behavior(self, context: Dict[str, Any], performance_data: Dict[str, Any] = None) -> Dict[str, Any]:
        """Dynamically adapt agent behavior based on context and performance"""

        adaptations_applied = {}

        # Get current performance metrics
        current_performance = performance_data or self.agent.get_expertise_assessment()

        # Apply performance-based adaptations
        for rule_name, rule in self.adaptation_rules.get('performance', {}).items():
            if rule['trigger'](current_performance):
                adaptations_applied[f"performance_{rule_name}"] = rule['adaptations']
                self._apply_trait_modifications(rule['adaptations'])

        # Apply context-based adaptations
        for rule_name, rule in self.adaptation_rules.get('context', {}).items():
            if rule['trigger'](context):
                adaptations_applied[f"context_{rule_name}"] = rule['adaptations']
                self._apply_trait_modifications(rule['adaptations'])

        # Apply domain-specific adaptations
        for rule_name, rule in self.adaptation_rules.get('domain', {}).items():
            if rule['trigger'](context):
                adaptations_applied[f"domain_{rule_name}"] = rule['adaptations']
                self._apply_trait_modifications(rule['adaptations'])

        # Record behavior change
        behavior_record = {
            'timestamp': time.time(),
            'context': context,
            'performance_data': current_performance,
            'adaptations_applied': adaptations_applied,
            'resulting_traits': {
                trait_id: {
                    'intensity': trait.intensity,
                    'active': trait.is_active(context)
                }
                for trait_id, trait in self.agent.traits.items()
            }
        }

        self.behavior_history.append(behavior_record)

        # Keep history manageable
        if len(self.behavior_history) > 50:
            self.behavior_history = self.behavior_history[-50:]

        print(f"   🎯 Applied {len(adaptations_applied)} behavioral adaptations")

        return {
            'adaptations_count': len(adaptations_applied),
            'adaptations_applied': adaptations_applied,
            'current_trait_states': behavior_record['resulting_traits']
        }

    def _apply_trait_modifications(self, adaptations: Dict[str, float]):
        """Apply trait modifications to the agent"""

        # Map adaptation types to trait modifications
        trait_mappings = {
            'increase_caution': ('decision_modifier', 0.1),
            'decrease_speed': ('speed_modifier', -0.1),
            'increase_detail_focus': ('decision_modifier', 0.15),
            'increase_confidence': ('decision_modifier', 0.1),
            'increase_speed': ('speed_modifier', 0.1),
            'increase_focus': ('decision_modifier', 0.2),
            'decrease_risk_tolerance': ('risk_modifier', -0.2),
            'increase_verification': ('speed_modifier', -0.1),
            'increase_communication': ('communication_modifier', 0.2),
            'increase_cooperation': ('decision_modifier', 0.1),
            'decrease_independence': ('decision_modifier', -0.1),
            'decrease_thoroughness': ('decision_modifier', -0.1),
            'focus_essentials': ('speed_modifier', 0.15)
        }

        for adaptation_type, intensity in adaptations.items():
            if adaptation_type in trait_mappings:
                trait_property, base_modifier = trait_mappings[adaptation_type]

                # Apply to relevant traits
                for trait in self.agent.traits.values():
                    if hasattr(trait, trait_property):
                        current_value = getattr(trait, trait_property)
                        new_value = current_value + (base_modifier * intensity)
                        # Keep within bounds
                        new_value = max(-1.0, min(1.0, new_value))
                        setattr(trait, trait_property, new_value)

    def _get_domain_specific_adaptations(self) -> Dict[str, Any]:
        """Get domain-specific adaptation rules"""

        domain_rules = {}

        if self.agent.domain.value == 'financial':
            domain_rules = {
                'market_volatility': {
                    'trigger': lambda context: context.get('market_volatility', 0) > 0.7,
                    'adaptations': {
                        'increase_caution': 0.3,
                        'decrease_risk_tolerance': 0.4,
                        'increase_verification': 0.2
                    }
                },
                'bull_market': {
                    'trigger': lambda context: context.get('market_direction') == 'bullish',
                    'adaptations': {
                        'increase_confidence': 0.15,
                        'increase_speed': 0.1
                    }
                }
            }

        elif self.agent.domain.value == 'healthcare':
            domain_rules = {
                'emergency_situation': {
                    'trigger': lambda context: context.get('emergency', False),
                    'adaptations': {
                        'increase_speed': 0.4,
                        'increase_focus': 0.3,
                        'decrease_risk_tolerance': 0.5
                    }
                },
                'patient_anxiety': {
                    'trigger': lambda context: context.get('patient_emotional_state') == 'anxious',
                    'adaptations': {
                        'increase_communication': 0.3,
                        'decrease_speed': 0.1
                    }
                }
            }

        elif self.agent.domain.value == 'legal':
            domain_rules = {
                'high_stakes_case': {
                    'trigger': lambda context: context.get('case_value', 0) > 1000000,
                    'adaptations': {
                        'increase_caution': 0.4,
                        'increase_detail_focus': 0.3,
                        'decrease_speed': 0.2
                    }
                },
                'litigation_risk': {
                    'trigger': lambda context: context.get('litigation_probability', 0) > 0.5,
                    'adaptations': {
                        'increase_verification': 0.3,
                        'decrease_risk_tolerance': 0.3
                    }
                }
            }

        return domain_rules

    def get_behavior_analytics(self) -> Dict[str, Any]:
        """Get analytics on behavior adaptation patterns"""

        if not self.behavior_history:
            return {'message': 'No behavior history available'}

        # Analyze adaptation frequency
        adaptation_frequency = defaultdict(int)
        context_triggers = defaultdict(int)

        for record in self.behavior_history:
            for adaptation_key in record['adaptations_applied'].keys():
                adaptation_frequency[adaptation_key] += 1

            # Track context patterns
            context = record['context']
            for key, value in context.items():
                if isinstance(value, (str, bool)):
                    context_triggers[f"{key}_{value}"] += 1
                elif isinstance(value, (int, float)) and value > 0:
                    context_triggers[key] += 1

        # Calculate adaptation effectiveness (simplified)
        recent_records = self.behavior_history[-10:]
        avg_success_rate = sum(
            record['performance_data'].get('success_rate', 0.5)
            for record in recent_records
        ) / len(recent_records) if recent_records else 0.5

        return {
            'total_adaptations': len(self.behavior_history),
            'most_frequent_adaptations': dict(sorted(
                adaptation_frequency.items(),
                key=lambda x: x[1],
                reverse=True
            )[:5]),
            'common_context_triggers': dict(sorted(
                context_triggers.items(),
                key=lambda x: x[1],
                reverse=True
            )[:5]),
            'recent_performance_trend': avg_success_rate,
            'adaptation_effectiveness': 'Good' if avg_success_rate > 0.7 else 'Needs Improvement'
        }

# =============================================================================
# PERSONALITY TEMPLATES - PREDEFINED BEHAVIORAL PROFILES
# =============================================================================

class PersonalityTemplate:
    """
    Predefined personality templates that can be applied to agents
    for consistent behavioral profiles across different scenarios.
    """

    @staticmethod
    def create_conservative_profile() -> Dict[str, AgentTrait]:
        """Create a conservative personality profile"""

        traits = {}

        # Risk Averse
        traits['risk_averse'] = AgentTrait(
            trait_id="risk_averse",
            name="Risk Averse",
            description="Strong preference for low-risk, proven approaches",
            intensity=0.8,
            decision_modifier=0.2,
            risk_modifier=-0.4,
            speed_modifier=-0.1
        )

        # Methodical
        traits['methodical'] = AgentTrait(
            trait_id="methodical",
            name="Methodical Approach",
            description="Systematic, step-by-step approach to problem solving",
            intensity=0.9,
            decision_modifier=0.3,
            speed_modifier=-0.2,
            communication_modifier=0.1
        )

        # Detail Oriented
        traits['detail_oriented'] = AgentTrait(
            trait_id="detail_oriented",
            name="Detail Oriented",
            description="Strong attention to details and accuracy",
            intensity=0.85,
            decision_modifier=0.25,
            speed_modifier=-0.15
        )

        return traits

    @staticmethod
    def create_aggressive_profile() -> Dict[str, AgentTrait]:
        """Create an aggressive personality profile"""

        traits = {}

        # Risk Seeking
        traits['risk_seeking'] = AgentTrait(
            trait_id="risk_seeking",
            name="Risk Seeking",
            description="Comfortable with high-risk, high-reward approaches",
            intensity=0.75,
            decision_modifier=-0.1,
            risk_modifier=0.4,
            speed_modifier=0.2
        )

        # Fast Decision Making
        traits['fast_decision'] = AgentTrait(
            trait_id="fast_decision",
            name="Fast Decision Making",
            description="Rapid decision making with available information",
            intensity=0.8,
            decision_modifier=-0.2,
            speed_modifier=0.3,
            communication_modifier=-0.1
        )

        # Opportunistic
        traits['opportunistic'] = AgentTrait(
            trait_id="opportunistic",
            name="Opportunistic",
            description="Quick to identify and act on opportunities",
            intensity=0.7,
            decision_modifier=0.1,
            speed_modifier=0.25,
            risk_modifier=0.2
        )

        return traits

    @staticmethod
    def create_balanced_profile() -> Dict[str, AgentTrait]:
        """Create a balanced personality profile"""

        traits = {}

        # Balanced Decision Making
        traits['balanced_decision'] = AgentTrait(
            trait_id="balanced_decision",
            name="Balanced Decision Making",
            description="Weighs risks and benefits carefully",
            intensity=0.7,
            decision_modifier=0.1,
            risk_modifier=0.0,
            speed_modifier=0.0
        )

        # Adaptive
        traits['adaptive'] = AgentTrait(
            trait_id="adaptive",
            name="Adaptive",
            description="Adjusts approach based on circumstances",
            intensity=0.8,
            decision_modifier=0.05,
            communication_modifier=0.1
        )

        # Collaborative
        traits['collaborative'] = AgentTrait(
            trait_id="collaborative",
            name="Collaborative",
            description="Works well with others and seeks input",
            intensity=0.75,
            communication_modifier=0.2,
            decision_modifier=0.1
        )

        return traits

    @staticmethod
    def create_analytical_profile() -> Dict[str, AgentTrait]:
        """Create an analytical personality profile"""

        traits = {}

        # Data Driven
        traits['data_driven'] = AgentTrait(
            trait_id="data_driven",
            name="Data Driven",
            description="Relies heavily on data and evidence for decisions",
            intensity=0.9,
            decision_modifier=0.3,
            speed_modifier=-0.1,
            communication_modifier=0.15
        )

        # Systematic
        traits['systematic'] = AgentTrait(
            trait_id="systematic",
            name="Systematic Analysis",
            description="Systematic approach to analysis and problem solving",
            intensity=0.85,
            decision_modifier=0.25,
            speed_modifier=-0.15
        )

        # Questioning
        traits['questioning'] = AgentTrait(
            trait_id="questioning",
            name="Questioning Nature",
            description="Asks probing questions and challenges assumptions",
            intensity=0.8,
            decision_modifier=0.2,
            communication_modifier=0.1,
            speed_modifier=-0.05
        )

        return traits

# =============================================================================
# AGENT COMPOSER - INTELLIGENT TEAM FORMATION
# =============================================================================

class AgentComposer:
    """
    Intelligent system for forming and managing teams of specialized agents.

    This class handles agent selection, team composition, task distribution,
    and coordination strategies for complex multi-agent workflows.
    """

    def __init__(self):
        self.available_agents = {}
        self.active_teams = {}
        self.composition_strategies = {}
        self.coordination_patterns = {}

        print("🔀 Agent Composer initialized")
        self.setup_composition_strategies()
        self.setup_coordination_patterns()

    def register_agent(self, agent: CustomAgentBase):
        """Register an agent as available for team composition"""

        self.available_agents[agent.agent_id] = {
            'agent': agent,
            'domain': agent.domain,
            'capabilities': list(agent.capabilities.keys()),
            'expertise_score': agent.get_expertise_assessment()['expertise_score'],
            'availability': True,
            'current_load': 0.0
        }

        print(f"   📝 Registered agent: {agent.name} ({agent.domain.value})")

    def compose_team(self, task_requirements: Dict[str, Any],
                    composition_strategy: str = "optimal_expertise") -> Dict[str, Any]:
        """Compose a team of agents for a given task"""

        if composition_strategy not in self.composition_strategies:
            return {
                'success': False,
                'error': f'Unknown composition strategy: {composition_strategy}',
                'available_strategies': list(self.composition_strategies.keys())
            }

        strategy_func = self.composition_strategies[composition_strategy]
        team_composition = strategy_func(task_requirements)

        if not team_composition['agents']:
            return {
                'success': False,
                'error': 'No suitable agents found for task requirements'
            }

        # Create team
        team_id = f"team_{int(time.time())}"
        team = {
            'team_id': team_id,
            'agents': team_composition['agents'],
            'task_requirements': task_requirements,
            'composition_strategy': composition_strategy,
            'coordination_pattern': task_requirements.get('coordination_pattern', 'parallel'),
            'created_at': time.time(),
            'status': 'formed'
        }

        self.active_teams[team_id] = team

        # Update agent loads
        for agent_info in team_composition['agents']:
            agent_id = agent_info['agent_id']
            if agent_id in self.available_agents:
                self.available_agents[agent_id]['current_load'] += agent_info.get('workload_allocation', 0.3)

        print(f"   🎯 Team '{team_id}' formed with {len(team_composition['agents'])} agents")

        return {
            'success': True,
            'team_id': team_id,
            'team_composition': team_composition,
            'coordination_pattern': team['coordination_pattern']
        }

    def execute_team_task(self, team_id: str, task_data: Dict[str, Any]) -> Dict[str, Any]:
        """Execute a task using the specified team"""

        if team_id not in self.active_teams:
            return {
                'success': False,
                'error': f'Team {team_id} not found'
            }

        team = self.active_teams[team_id]
        coordination_pattern = team['coordination_pattern']

        if coordination_pattern not in self.coordination_patterns:
            return {
                'success': False,
                'error': f'Unknown coordination pattern: {coordination_pattern}'
            }

        coordination_func = self.coordination_patterns[coordination_pattern]
        execution_result = coordination_func(team, task_data)

        # Update team status
        team['status'] = 'completed' if execution_result['success'] else 'failed'
        team['last_execution'] = time.time()

        return execution_result

    def setup_composition_strategies(self):
        """Setup different team composition strategies"""

        def optimal_expertise_strategy(task_requirements: Dict[str, Any]) -> Dict[str, Any]:
            """Compose team based on optimal expertise for required capabilities"""

            required_capabilities = task_requirements.get('required_capabilities', [])
            required_domains = task_requirements.get('required_domains', [])
            team_size_limit = task_requirements.get('max_team_size', 5)

            candidate_agents = []

            # Score agents based on requirement match
            for agent_id, agent_info in self.available_agents.items():
                if not agent_info['availability'] or agent_info['current_load'] > 0.8:
                    continue

                score = 0.0

                # Domain match score
                if required_domains and agent_info['domain'].value in required_domains:
                    score += 0.4

                # Capability match score
                if required_capabilities:
                    capability_matches = sum(
                        1 for cap in required_capabilities
                        if cap in agent_info['capabilities']
                    )
                    score += (capability_matches / len(required_capabilities)) * 0.4

                # Expertise score
                score += agent_info['expertise_score'] * 0.2

                if score > 0:
                    candidate_agents.append({
                        'agent_id': agent_id,
                        'agent': agent_info['agent'],
                        'score': score,
                        'domain': agent_info['domain'].value,
                        'capabilities': agent_info['capabilities'],
                        'workload_allocation': 1.0 / min(team_size_limit, len(required_capabilities) or 1)
                    })

            # Sort by score and select top agents
            candidate_agents.sort(key=lambda x: x['score'], reverse=True)
            selected_agents = candidate_agents[:team_size_limit]

            return {
                'agents': selected_agents,
                'composition_rationale': f'Selected {len(selected_agents)} agents based on expertise match',
                'total_capability_coverage': sum(agent['score'] for agent in selected_agents) / len(selected_agents) if selected_agents else 0
            }

        def diverse_domains_strategy(task_requirements: Dict[str, Any]) -> Dict[str, Any]:
            """Compose team with maximum domain diversity"""

            team_size_limit = task_requirements.get('max_team_size', 4)

            # Group agents by domain
            agents_by_domain = defaultdict(list)
            for agent_id, agent_info in self.available_agents.items():
                if agent_info['availability'] and agent_info['current_load'] < 0.8:
                    agents_by_domain[agent_info['domain'].value].append({
                        'agent_id': agent_id,
                        'agent': agent_info['agent'],
                        'domain': agent_info['domain'].value,
                        'expertise_score': agent_info['expertise_score'],
                        'capabilities': agent_info['capabilities']
                    })

            # Select best agent from each domain
            selected_agents = []
            for domain, domain_agents in agents_by_domain.items():
                if len(selected_agents) >= team_size_limit:
                    break

                # Select highest expertise agent from domain
                best_agent = max(domain_agents, key=lambda x: x['expertise_score'])
                best_agent['workload_allocation'] = 1.0 / team_size_limit
                selected_agents.append(best_agent)

            return {
                'agents': selected_agents,
                'composition_rationale': f'Selected for maximum domain diversity: {[a["domain"] for a in selected_agents]}',
                'domain_coverage': len(set(agent['domain'] for agent in selected_agents))
            }

        def specialist_strategy(task_requirements: Dict[str, Any]) -> Dict[str, Any]:
            """Compose team focused on a single domain with deep specialization"""

            target_domain = task_requirements.get('target_domain')
            if not target_domain:
                return {'agents': [], 'composition_rationale': 'No target domain specified'}

            team_size_limit = task_requirements.get('max_team_size', 3)

            # Find all agents in target domain
            domain_agents = []
            for agent_id, agent_info in self.available_agents.items():
                if (agent_info['availability'] and
                    agent_info['current_load'] < 0.8 and
                    agent_info['domain'].value == target_domain):

                    domain_agents.append({
                        'agent_id': agent_id,
                        'agent': agent_info['agent'],
                        'domain': agent_info['domain'].value,
                        'expertise_score': agent_info['expertise_score'],
                        'capabilities': agent_info['capabilities'],
                        'workload_allocation': 1.0 / min(team_size_limit, len(domain_agents) + 1)
                    })

            # Select top specialists
            domain_agents.sort(key=lambda x: x['expertise_score'], reverse=True)
            selected_agents = domain_agents[:team_size_limit]

            return {
                'agents': selected_agents,
                'composition_rationale': f'Deep specialization in {target_domain}',
                'average_expertise': sum(a['expertise_score'] for a in selected_agents) / len(selected_agents) if selected_agents else 0
            }

        self.composition_strategies = {
            'optimal_expertise': optimal_expertise_strategy,
            'diverse_domains': diverse_domains_strategy,
            'specialist': specialist_strategy
        }

    def setup_coordination_patterns(self):
        """Setup different team coordination patterns"""

        def parallel_coordination(team: Dict, task_data: Dict[str, Any]) -> Dict[str, Any]:
            """Execute tasks in parallel across all team members"""

            agents = team['agents']
            results = {}

            print(f"   🔄 Executing parallel coordination with {len(agents)} agents")

            # Execute task with each agent simultaneously (simulated)
            for agent_info in agents:
                agent = agent_info['agent']

                # Determine operation based on agent capabilities and task
                operation_type = self._determine_agent_operation(agent, task_data)

                try:
                    result = agent.specialized_operation(operation_type, **task_data)
                    results[agent.agent_id] = {
                        'agent_name': agent.name,
                        'operation_type': operation_type,
                        'result': result,
                        'execution_time': time.time()
                    }
                except Exception as e:
                    results[agent.agent_id] = {
                        'agent_name': agent.name,
                        'operation_type': operation_type,
                        'error': str(e),
                        'execution_time': time.time()
                    }

            # Aggregate results
            successful_results = [r for r in results.values() if 'result' in r and r['result'].get('success')]

            return {
                'success': len(successful_results) > 0,
                'coordination_pattern': 'parallel',
                'individual_results': results,
                'aggregated_insights': self._aggregate_parallel_results(successful_results),
                'team_performance': {
                    'total_agents': len(agents),
                    'successful_executions': len(successful_results),
                    'success_rate': len(successful_results) / len(agents) if agents else 0
                }
            }

        def sequential_coordination(team: Dict, task_data: Dict[str, Any]) -> Dict[str, Any]:
            """Execute tasks sequentially, passing results between agents"""

            agents = team['agents']
            results = []
            current_data = task_data.copy()

            print(f"   🔄 Executing sequential coordination with {len(agents)} agents")

            for i, agent_info in enumerate(agents):
                agent = agent_info['agent']
                operation_type = self._determine_agent_operation(agent, current_data)

                try:
                    result = agent.specialized_operation(operation_type, **current_data)

                    step_result = {
                        'step': i + 1,
                        'agent_name': agent.name,
                        'agent_domain': agent.domain.value,
                        'operation_type': operation_type,
                        'result': result,
                        'execution_time': time.time()
                    }

                    results.append(step_result)

                    # Pass successful results to next agent
                    if result.get('success'):
                        current_data.update({
                            f'previous_step_result': result,
                            f'step_{i}_output': result
                        })
                    else:
                        # Stop on failure unless configured otherwise
                        break

                except Exception as e:
                    results.append({
                        'step': i + 1,
                        'agent_name': agent.name,
                        'error': str(e),
                        'execution_time': time.time()
                    })
                    break

            return {
                'success': len(results) > 0 and results[-1].get('result', {}).get('success', False),
                'coordination_pattern': 'sequential',
                'execution_steps': results,
                'final_result': results[-1] if results else None,
                'pipeline_efficiency': len([r for r in results if 'result' in r]) / len(agents) if agents else 0
            }

        def hierarchical_coordination(team: Dict, task_data: Dict[str, Any]) -> Dict[str, Any]:
            """Execute with hierarchical coordination - lead agent coordinating others"""

            agents = team['agents']
            if not agents:
                return {'success': False, 'error': 'No agents in team'}

            # Select lead agent (highest expertise score)
            lead_agent_info = max(agents, key=lambda x: x.get('expertise_score', 0))
            subordinate_agents = [a for a in agents if a != lead_agent_info]

            print(f"   🔄 Executing hierarchical coordination: {lead_agent_info['agent'].name} leading {len(subordinate_agents)} agents")

            # Subordinate agents provide input
            subordinate_results = {}
            for agent_info in subordinate_agents:
                agent = agent_info['agent']
                operation_type = self._determine_agent_operation(agent, task_data)

                try:
                    result = agent.specialized_operation(operation_type, **task_data)
                    subordinate_results[agent.agent_id] = {
                        'agent_name': agent.name,
                        'domain': agent.domain.value,
                        'input': result
                    }
                except Exception as e:
                    subordinate_results[agent.agent_id] = {
                        'agent_name': agent.name,
                        'domain': agent.domain.value,
                        'error': str(e)
                    }

            # Lead agent makes final decision incorporating subordinate input
            lead_task_data = task_data.copy()
            lead_task_data['subordinate_inputs'] = subordinate_results

            lead_agent = lead_agent_info['agent']
            lead_operation = self._determine_agent_operation(lead_agent, lead_task_data)

            try:
                final_result = lead_agent.specialized_operation(lead_operation, **lead_task_data)

                return {
                    'success': final_result.get('success', False),
                    'coordination_pattern': 'hierarchical',
                    'lead_agent': lead_agent_info['agent'].name,
                    'subordinate_inputs': subordinate_results,
                    'final_decision': final_result,
                    'team_consensus': self._calculate_team_consensus(subordinate_results, final_result)
                }

            except Exception as e:
                return {
                    'success': False,
                    'coordination_pattern': 'hierarchical',
                    'lead_agent': lead_agent_info['agent'].name,
                    'error': f'Lead agent execution failed: {str(e)}'
                }

        self.coordination_patterns = {
            'parallel': parallel_coordination,
            'sequential': sequential_coordination,
            'hierarchical': hierarchical_coordination
        }

    def _determine_agent_operation(self, agent: CustomAgentBase, task_data: Dict[str, Any]) -> str:
        """Determine appropriate operation for agent based on task and capabilities"""

        # Map task types to agent operations
        task_type = task_data.get('task_type', 'general_analysis')

        # Domain-specific operation mapping
        domain_operations = {
            'financial': {
                'analysis': 'complete_portfolio_analysis',
                'prediction': 'market_outlook',
                'recommendation': 'investment_recommendation',
                'planning': 'financial_planning'
            },
            'healthcare': {
                'assessment': 'comprehensive_health_assessment',
                'consultation': 'medication_consultation',
                'planning': 'wellness_planning',
                'education': 'health_education'
            },
            'legal': {
                'review': 'comprehensive_contract_review',
                'compliance': 'legal_compliance_assessment',
                'strategy': 'legal_strategy_development',
                'planning': 'risk_mitigation_planning'
            }
        }

        domain_ops = domain_operations.get(agent.domain.value, {})
        return domain_ops.get(task_type, list(domain_ops.values())[0] if domain_ops else 'specialized_operation')

    def _aggregate_parallel_results(self, successful_results: List[Dict]) -> Dict[str, Any]:
        """Aggregate results from parallel execution"""

        if not successful_results:
            return {
                'message': 'No successful results to aggregate',
                'total_insights': 0,
                'consensus_insights': [],
                'average_confidence': 0.0,
                'result_diversity': 0,
                'team_agreement_level': 0.0
            }

        # Extract common insights
        all_insights = []
        confidence_scores = []
        recommendations = []

        for result_data in successful_results:
            result = result_data['result']

            # Collect insights from different result structures
            if isinstance(result, dict):
                # Extract insights based on result structure
                if 'analysis_report' in result:
                    insights = result['analysis_report'].get('recommendations', [])
                    all_insights.extend(insights)

                if 'health_assessment' in result:
                    insights = result['health_assessment'].get('health_recommendations', [])
                    all_insights.extend(insights)

                if 'contract_review' in result:
                    insights = result['contract_review'].get('detailed_recommendations', [])
                    all_insights.extend(insights)

                # Extract confidence/quality scores
                confidence = result.get('analyst_confidence', result.get('confidence', 0.7))
                confidence_scores.append(confidence)

        # Generate consensus insights
        insight_frequency = defaultdict(int)
        for insight in all_insights:
            if isinstance(insight, str):
                insight_frequency[insight] += 1

        consensus_insights = [
            insight for insight, count in insight_frequency.items()
            if count > len(successful_results) / 2  # Majority consensus
        ]

        return {
            'total_insights': len(all_insights),
            'consensus_insights': consensus_insights,
            'average_confidence': sum(confidence_scores) / len(confidence_scores) if confidence_scores else 0.5,
            'result_diversity': len(set(str(r['result']) for r in successful_results)),
            'team_agreement_level': len(consensus_insights) / max(len(all_insights), 1)
        }

        if not successful_results:
            return {'message': 'No successful results to aggregate'}

        # Extract common insights
        all_insights = []
        confidence_scores = []
        recommendations = []

        for result_data in successful_results:
            result = result_data['result']

            # Collect insights from different result structures
            if isinstance(result, dict):
                # Extract insights based on result structure
                if 'analysis_report' in result:
                    insights = result['analysis_report'].get('recommendations', [])
                    all_insights.extend(insights)

                if 'health_assessment' in result:
                    insights = result['health_assessment'].get('health_recommendations', [])
                    all_insights.extend(insights)

                if 'contract_review' in result:
                    insights = result['contract_review'].get('detailed_recommendations', [])
                    all_insights.extend(insights)

                # Extract confidence/quality scores
                confidence = result.get('analyst_confidence', result.get('confidence', 0.7))
                confidence_scores.append(confidence)

        # Generate consensus insights
        insight_frequency = defaultdict(int)
        for insight in all_insights:
            if isinstance(insight, str):
                insight_frequency[insight] += 1

        consensus_insights = [
            insight for insight, count in insight_frequency.items()
            if count > len(successful_results) / 2  # Majority consensus
        ]

        return {
            'total_insights': len(all_insights),
            'consensus_insights': consensus_insights,
            'average_confidence': sum(confidence_scores) / len(confidence_scores) if confidence_scores else 0.5,
            'result_diversity': len(set(str(r['result']) for r in successful_results)),
            'team_agreement_level': len(consensus_insights) / max(len(all_insights), 1)
        }

    def _calculate_team_consensus(self, subordinate_results: Dict, final_result: Dict) -> float:
        """Calculate consensus level between team members"""

        if not subordinate_results:
            return 1.0

        # Simple consensus calculation based on successful vs failed subordinate inputs
        successful_inputs = sum(
            1 for result_data in subordinate_results.values()
            if 'input' in result_data and result_data['input'].get('success', False)
        )

        total_inputs = len(subordinate_results)
        base_consensus = successful_inputs / total_inputs if total_inputs > 0 else 0

        # Adjust based on final result success
        if final_result.get('success', False):
            return min(1.0, base_consensus + 0.2)
        else:
            return max(0.0, base_consensus - 0.2)

    def get_team_analytics(self) -> Dict[str, Any]:
        """Get analytics on team composition and performance"""

        if not self.active_teams:
            return {'message': 'No active teams to analyze'}

        # Team composition analysis
        total_teams = len(self.active_teams)
        completed_teams = sum(1 for team in self.active_teams.values() if team['status'] == 'completed')

        # Strategy usage
        strategy_usage = defaultdict(int)
        coordination_usage = defaultdict(int)

        for team in self.active_teams.values():
            strategy_usage[team['composition_strategy']] += 1
            coordination_usage[team['coordination_pattern']] += 1

        # Agent utilization
        agent_utilization = {
            agent_id: agent_info['current_load']
            for agent_id, agent_info in self.available_agents.items()
        }

        return {
            'total_teams_formed': total_teams,
            'completion_rate': completed_teams / total_teams if total_teams > 0 else 0,
            'strategy_preferences': dict(strategy_usage),
            'coordination_preferences': dict(coordination_usage),
            'agent_utilization': agent_utilization,
            'high_utilization_agents': [
                agent_id for agent_id, load in agent_utilization.items() if load > 0.7
            ]
        }

# =============================================================================
# DEMONSTRATION AND TESTING
# =============================================================================

print("🧪 Testing Dynamic Composition & Behavior Control...")
print()

# Create specialized agents for testing
print("🏭 Creating specialized agents for team composition:")

# Create agents with different personalities
financial_agent_conservative = FinancialAnalystAgent("ConservativeFA", risk_tolerance="conservative")
financial_agent_aggressive = FinancialAnalystAgent("AggressiveFA", risk_tolerance="aggressive")
health_agent = HealthcareAdvisorAgent("HealthExpert", specialty="general")
legal_agent = LegalAdvisorAgent("LegalExpert", practice_area="contract")

# Apply personality templates
print("\n🎭 Applying personality templates:")

# Apply conservative template to one financial agent
conservative_traits = PersonalityTemplate.create_conservative_profile()
for trait in conservative_traits.values():
    financial_agent_conservative.add_trait(trait)
print(f"   Applied conservative profile to {financial_agent_conservative.name}")

# Apply aggressive template to another financial agent
aggressive_traits = PersonalityTemplate.create_aggressive_profile()
for trait in aggressive_traits.values():
    financial_agent_aggressive.add_trait(trait)
print(f"   Applied aggressive profile to {financial_agent_aggressive.name}")

# Apply analytical template to healthcare agent
analytical_traits = PersonalityTemplate.create_analytical_profile()
for trait in analytical_traits.values():
    health_agent.add_trait(trait)
print(f"   Applied analytical profile to {health_agent.name}")

# Apply balanced template to legal agent
balanced_traits = PersonalityTemplate.create_balanced_profile()
for trait in balanced_traits.values():
    legal_agent.add_trait(trait)
print(f"   Applied balanced profile to {legal_agent.name}")

# Test Behavior Controller
print("\n🎯 Testing Behavior Controller:")
behavior_controller = BehaviorController(financial_agent_conservative)

# Test behavior adaptation
test_context = {
    'market_volatility': 0.8,
    'urgency': 'high',
    'team_size': 1,
    'deadline_pressure': 0.9
}

adaptation_result = behavior_controller.adapt_behavior(test_context)
print(f"   Applied {adaptation_result['adaptations_count']} behavioral adaptations")
print(f"   Adaptations: {list(adaptation_result['adaptations_applied'].keys())}")

# Test Agent Composer
print("\n🔀 Testing Agent Composer:")
composer = AgentComposer()

# Register agents
composer.register_agent(financial_agent_conservative)
composer.register_agent(financial_agent_aggressive)
composer.register_agent(health_agent)
composer.register_agent(legal_agent)

# Test team composition strategies
print("\n   Testing composition strategies:")

# 1. Optimal expertise strategy
task_requirements_1 = {
    'required_capabilities': ['financial_analysis', 'risk_assessment'],
    'required_domains': ['financial'],
    'max_team_size': 2,
    'coordination_pattern': 'parallel'
}

team_result_1 = composer.compose_team(task_requirements_1, "optimal_expertise")
if team_result_1['success']:
    print(f"   ✅ Optimal expertise team: {len(team_result_1['team_composition']['agents'])} agents")
    for agent in team_result_1['team_composition']['agents']:
        print(f"      - {agent['agent'].name} ({agent['domain']}) - Score: {agent['score']:.2f}")

# 2. Diverse domains strategy
task_requirements_2 = {
    'max_team_size': 4,
    'coordination_pattern': 'hierarchical'
}

team_result_2 = composer.compose_team(task_requirements_2, "diverse_domains")
if team_result_2['success']:
    print(f"   ✅ Diverse domains team: {len(team_result_2['team_composition']['agents'])} agents")
    domains = [agent['domain'] for agent in team_result_2['team_composition']['agents']]
    print(f"      Domains covered: {', '.join(set(domains))}")

# Test team execution
print("\n   Testing team coordination patterns:")

# Test parallel coordination
if team_result_1['success']:
    task_data = {
        'task_type': 'analysis',
        'portfolio_data': {
            'holdings': [
                {'symbol': 'AAPL', 'value': 100000, 'beta': 1.2},
                {'symbol': 'BONDS', 'value': 50000, 'beta': 0.2}
            ]
        }
    }

    execution_result = composer.execute_team_task(team_result_1['team_id'], task_data)
    if execution_result['success']:
        team_perf = execution_result.get('team_performance', {})
        success_rate = team_perf.get('success_rate', 0)
        print(f"   ✅ Parallel coordination: {success_rate:.1%} success rate")

        insights = execution_result.get('aggregated_insights', {})
        consensus_insights = insights.get('consensus_insights', [])
        print(f"      Team insights: {len(consensus_insights)} consensus points")
    else:
        print(f"   ❌ Parallel coordination failed: {execution_result.get('error', 'Unknown error')}")

# Test sequential coordination
task_requirements_3 = {
    'required_domains': ['financial', 'legal'],
    'max_team_size': 2,
    'coordination_pattern': 'sequential'
}

team_result_3 = composer.compose_team(task_requirements_3, "optimal_expertise")
if team_result_3['success']:
    task_data_sequential = {
        'task_type': 'review',
        'contract_data': {
            'contract_text': 'Investment advisory agreement with performance fees...',
            'type': 'advisory_agreement'
        }
    }

    execution_result_seq = composer.execute_team_task(team_result_3['team_id'], task_data_sequential)
    if execution_result_seq['success']:
        execution_steps = execution_result_seq.get('execution_steps', [])
        pipeline_efficiency = execution_result_seq.get('pipeline_efficiency', 0)
        print(f"   ✅ Sequential coordination: {len(execution_steps)} steps completed")
        print(f"      Pipeline efficiency: {pipeline_efficiency:.1%}")
    else:
        print(f"   ❌ Sequential coordination failed: {execution_result_seq.get('error', 'Unknown error')}")

# Test behavior analytics
print("\n📊 Behavior Analytics:")
behavior_analytics = behavior_controller.get_behavior_analytics()
if 'total_adaptations' in behavior_analytics:
    print(f"   Total behavioral adaptations: {behavior_analytics['total_adaptations']}")
    print(f"   Recent performance trend: {behavior_analytics['recent_performance_trend']:.2f}")
    print(f"   Adaptation effectiveness: {behavior_analytics['adaptation_effectiveness']}")

# Test team analytics
team_analytics = composer.get_team_analytics()
if 'total_teams_formed' in team_analytics:
    print(f"   Total teams formed: {team_analytics['total_teams_formed']}")
    print(f"   Team completion rate: {team_analytics['completion_rate']:.1%}")
    print(f"   Most used strategy: {max(team_analytics['strategy_preferences'].items(), key=lambda x: x[1])[0]}")

print()

print("🎉 Dynamic Composition & Behavior Control working correctly!")

print()

print("📋 Summary of Part 3:")

print("   • BehaviorController - Dynamic personality adaptation based on context and performance")

print("   • PersonalityTemplate - Predefined behavioral profiles (conservative, aggressive, balanced, analytical)")

print("   • AgentComposer - Intelligent team formation with multiple composition strategies")

print("   • Coordination Patterns - Parallel, sequential, and hierarchical team execution")

print("   • Performance Analytics - Behavior adaptation tracking and team performance metrics")

print("   • Context-Aware Adaptation - Real-time behavioral modification based on:")

print("     - Performance feedback")

print("     - Environmental context")

print("     - Domain-specific conditions")

print("     - Team dynamics")

print()

print("   Ready for Complete Summary & Error Analysis")


#This completes Tutorial 14 Part 3: Dynamic Composition & Behavior Control

#What we built:

# - BehaviorController for dynamic personality adaptation

# - PersonalityTemplate system for consistent behavioral profiles

# - AgentComposer for intelligent team formation and management

# - Multiple coordination patterns (parallel, sequential, hierarchical)

# - Context-aware behavior modification systems

# - Comprehensive analytics for behavior and team performance

# - Real-time adaptation based on performance feedback and environmental factors

# The system now supports:

# - Individual agent behavior modification

# - Intelligent team composition based on task requirements

# - Multiple team coordination strategies

# - Performance-driven adaptation

# - Context-sensitive behavioral changes

# - Comprehensive monitoring and analytics


🧪 Testing Dynamic Composition & Behavior Control...

🏭 Creating specialized agents for team composition:
📚 Domain knowledge initialized for: financial
🤖 Custom agent created: ConservativeFA
   Domain: financial
   Agent ID: 9fcea3d5...
   ➕ Added capability: Portfolio Analysis (ADVANCED)
   ➕ Added capability: Market Prediction (ADVANCED)
   ➕ Added capability: Investment Risk Assessment (MASTER)
   🎭 Added trait: Conservative Analysis (intensity: 0.80)
   🎭 Added trait: Detail-Oriented Analysis (intensity: 0.90)
   🎭 Added trait: Market Awareness (intensity: 0.85)
   📝 Added fact: Compound interest is interest calculated on the in...
   📝 Added fact: Diversification reduces risk by spreading investme...
   📝 Added fact: Higher potential returns generally come with highe...
   📋 Added rule: If investor age > 30 and risk_tolerance = conservative → Allocate maximum 60% to equities, 40% to bonds and cash
   📋 Added rule: If emergency_fund < 6_months_expenses → Prioritize building emergenc

In [18]:
"""
Tutorial 14: Framework Summary & Architectural Insights
=======================================================

Comprehensive overview of the Custom Agent Types framework with
architectural insights, design patterns, and learning outcomes.

This covers the complete framework built across Parts 1-3.
"""

import json
from typing import Dict, List, Any
from datetime import datetime

print("Tutorial 14: Framework Summary & Architectural Insights")
print("=" * 55)
print("Complete framework overview and design insights...")
print()

# =============================================================================
# FRAMEWORK OVERVIEW
# =============================================================================

class FrameworkSummary:
    """
    Complete summary of Tutorial 14 framework components and achievements
    """

    def __init__(self):
        self.framework_components = self._build_framework_overview()
        self.learning_outcomes = self._define_learning_outcomes()
        self.architectural_insights = self._capture_architectural_insights()
        self.design_patterns = self._identify_design_patterns()
        self.use_cases = self._define_use_cases()

    def _build_framework_overview(self) -> Dict[str, Any]:
        """Build comprehensive overview of framework components"""

        return {
            "part_1_foundation": {
                "description": "Core agent specialization framework and building blocks",
                "key_classes": {
                    "AgentCapability": {
                        "purpose": "Executable skills with performance tracking and resource management",
                        "features": [
                            "Performance metrics (accuracy, speed, reliability)",
                            "Cooldown and resource cost management",
                            "Usage tracking and success rate calculation",
                            "Execution function with error handling"
                        ]
                    },
                    "AgentTrait": {
                        "purpose": "Behavioral modifiers affecting decision-making and communication",
                        "features": [
                            "Context-aware activation and deactivation",
                            "Intensity scaling for fine-grained control",
                            "Multiple modifier types (decision, risk, speed, communication)",
                            "Flexible trait combination and composition"
                        ]
                    },
                    "DomainKnowledge": {
                        "purpose": "Expert knowledge bases with structured information storage",
                        "features": [
                            "Facts, rules, procedures, and terminology storage",
                            "Confidence scoring for uncertainty handling",
                            "Queryable interface with relevance scoring",
                            "Statistical analysis and coverage reporting"
                        ]
                    },
                    "CustomAgentBase": {
                        "purpose": "Abstract foundation providing common agent functionality",
                        "features": [
                            "Capability and trait management",
                            "Context-aware operation execution",
                            "Performance assessment and expertise scoring",
                            "Execution history tracking and analysis"
                        ]
                    }
                },
                "design_principles": [
                    "Modular composition over inheritance",
                    "Performance-driven adaptation mechanisms",
                    "Context-sensitive behavioral activation",
                    "Comprehensive monitoring and analytics"
                ]
            },

            "part_2_specialization": {
                "description": "Domain-specific agent implementations with professional expertise",
                "implemented_agents": {
                    "FinancialAnalystAgent": {
                        "domain": "Financial analysis and advisory services",
                        "capabilities": [
                            "Portfolio analysis with risk assessment",
                            "Market prediction and trend analysis",
                            "Investment recommendation generation",
                            "Financial planning and goal setting"
                        ],
                        "traits": [
                            "Conservative analysis (risk-averse decision making)",
                            "Detail-oriented approach (thorough analysis)",
                            "Market awareness (economic sensitivity)"
                        ],
                        "safety_features": [
                            "Risk tolerance configuration",
                            "Investment disclaimers and warnings",
                            "Professional consultation recommendations"
                        ]
                    },
                    "HealthcareAdvisorAgent": {
                        "domain": "Healthcare guidance and medical information",
                        "capabilities": [
                            "Symptom analysis with severity assessment",
                            "Health information provision and education",
                            "Emergency triage and priority determination",
                            "Medication consultation and interaction checking"
                        ],
                        "traits": [
                            "Safety-first approach (conservative medical guidance)",
                            "Empathetic communication (patient-centered)",
                            "Evidence-based practice (scientific rigor)"
                        ],
                        "safety_features": [
                            "Medical disclaimers and professional referrals",
                            "Emergency protocol activation",
                            "Conservative severity assessment"
                        ]
                    },
                    "LegalAdvisorAgent": {
                        "domain": "Legal research and document analysis",
                        "capabilities": [
                            "Contract analysis and risk identification",
                            "Legal research and case law analysis",
                            "Document drafting and template generation",
                            "Compliance assessment and recommendations"
                        ],
                        "traits": [
                            "Precision-focused analysis (detailed examination)",
                            "Conservative advisory approach (risk management)",
                            "Professional disclaimer emphasis (legal boundaries)"
                        ],
                        "safety_features": [
                            "Legal information vs advice distinctions",
                            "Professional legal counsel recommendations",
                            "Compliance and regulatory awareness"
                        ]
                    }
                },
                "domain_expertise_features": [
                    "Professional knowledge bases with industry standards",
                    "Domain-specific behavioral traits and decision patterns",
                    "Specialized operational workflows and procedures",
                    "Comprehensive safety protocols and ethical guidelines"
                ]
            },

            "part_3_composition": {
                "description": "Dynamic behavior control and intelligent team formation",
                "behavioral_systems": {
                    "BehaviorController": {
                        "purpose": "Dynamic personality adaptation based on context and performance",
                        "features": [
                            "Performance-based behavior modification",
                            "Context-sensitive trait adjustment",
                            "Domain-specific adaptation rules",
                            "Behavioral history tracking and analytics"
                        ]
                    },
                    "PersonalityTemplate": {
                        "purpose": "Predefined behavioral profiles for consistent agent personalities",
                        "templates": [
                            "Conservative (risk-averse, methodical, detail-oriented)",
                            "Aggressive (risk-seeking, fast decisions, opportunistic)",
                            "Balanced (measured decisions, adaptive, collaborative)",
                            "Analytical (data-driven, systematic, questioning)"
                        ]
                    }
                },
                "team_coordination": {
                    "AgentComposer": {
                        "purpose": "Intelligent team formation and management system",
                        "composition_strategies": [
                            "Optimal expertise (capability and expertise matching)",
                            "Diverse domains (maximum domain coverage)",
                            "Specialist focus (deep domain specialization)"
                        ],
                        "coordination_patterns": [
                            "Parallel (simultaneous independent execution)",
                            "Sequential (pipeline with result passing)",
                            "Hierarchical (lead agent with subordinate input)"
                        ]
                    }
                },
                "adaptation_mechanisms": [
                    "Real-time behavior modification based on performance feedback",
                    "Context-aware trait activation and intensity adjustment",
                    "Load balancing and resource management across team members",
                    "Comprehensive team performance analytics and optimization"
                ]
            }
        }

    def _define_learning_outcomes(self) -> List[str]:
        """Define key learning outcomes from the tutorial"""

        return [
            "🏗️ ARCHITECTURAL MASTERY",
            "  • Designed modular agent capability and trait composition systems",
            "  • Implemented context-aware behavioral adaptation mechanisms",
            "  • Built intelligent team formation with multiple coordination strategies",
            "  • Created comprehensive performance monitoring and analytics systems",
            "",
            "🎯 DOMAIN EXPERTISE IMPLEMENTATION",
            "  • Financial analysis with portfolio optimization and risk management",
            "  • Healthcare guidance with safety protocols and emergency assessment",
            "  • Legal analysis with compliance awareness and professional boundaries",
            "  • Cross-domain knowledge integration and team collaboration patterns",
            "",
            "🔄 DYNAMIC SYSTEM DESIGN",
            "  • Real-time personality adaptation based on context and performance",
            "  • Multiple team composition strategies for different task scenarios",
            "  • Coordinated multi-agent workflows with intelligent load balancing",
            "  • Behavioral analytics and adaptation effectiveness measurement",
            "",
            "🛡️ PRODUCTION READINESS PRINCIPLES",
            "  • Professional safety considerations and ethical guideline implementation",
            "  • Comprehensive error handling and graceful degradation patterns",
            "  • Performance optimization strategies and scalability design principles",
            "  • Monitoring, debugging, and troubleshooting capability integration"
        ]

    def _capture_architectural_insights(self) -> Dict[str, List[str]]:
        """Capture key architectural insights and design principles"""

        return {
            "capability_architecture": [
                "Capabilities as composable, executable units with intrinsic performance metrics",
                "Cooldown mechanisms prevent capability abuse and simulate realistic resource constraints",
                "Success rate tracking enables automatic performance-based behavioral adaptations",
                "Resource cost modeling allows for intelligent load balancing and team optimization",
                "Validation functions provide quality control and input sanitization"
            ],

            "trait_system_design": [
                "Traits as orthogonal behavioral modifiers enabling flexible personality composition",
                "Context-sensitive activation prevents inappropriate behavioral responses in different scenarios",
                "Intensity scaling provides fine-grained control over behavioral influence magnitude",
                "Modifier application to base values enables predictable and measurable behavioral changes",
                "Trait conflict resolution through weighted averaging based on trait intensity"
            ],

            "knowledge_representation": [
                "Structured knowledge organization with facts, rules, procedures, and terminology",
                "Confidence scoring for knowledge items enables sophisticated uncertainty handling",
                "Queryable knowledge base with relevance scoring for intelligent information retrieval",
                "Domain-specific knowledge organization following professional and industry standards",
                "Knowledge consistency validation and contradiction detection mechanisms"
            ],

            "team_coordination_architecture": [
                "Multiple coordination patterns handle different collaboration and workflow scenarios",
                "Agent composition strategies optimize team formation for specific task requirements",
                "Load balancing mechanisms prevent individual agent overutilization and bottlenecks",
                "Result aggregation algorithms synthesize insights and consensus from multiple agents",
                "Fallback strategies ensure robust team formation even with constrained resources"
            ],

            "behavioral_adaptation_system": [
                "Performance feedback loops enable continuous behavioral improvement and optimization",
                "Context-aware adaptations respond dynamically to environmental and task changes",
                "Domain-specific adaptation rules handle specialized scenarios and requirements",
                "Adaptation history tracking provides behavioral analytics and debugging capabilities",
                "Damping mechanisms prevent behavioral oscillation and ensure stable adaptations"
            ]
        }

    def _identify_design_patterns(self) -> Dict[str, Dict[str, Any]]:
        """Identify key design patterns used in the framework"""

        return {
            "composition_over_inheritance": {
                "description": "Agents built through capability and trait composition rather than inheritance",
                "benefits": [
                    "Flexible agent construction with mix-and-match components",
                    "Runtime modification of agent behavior and capabilities",
                    "Easier testing and debugging of individual components",
                    "Reduced coupling and improved maintainability"
                ],
                "implementation": "AgentCapability and AgentTrait composition in CustomAgentBase"
            },

            "strategy_pattern": {
                "description": "Multiple algorithms for team composition and coordination",
                "benefits": [
                    "Pluggable composition strategies for different scenarios",
                    "Easy addition of new coordination patterns",
                    "Runtime strategy selection based on requirements",
                    "Isolated testing of individual strategies"
                ],
                "implementation": "AgentComposer with composition_strategies and coordination_patterns"
            },

            "observer_pattern": {
                "description": "Performance monitoring and behavioral adaptation based on observations",
                "benefits": [
                    "Decoupled monitoring from core agent functionality",
                    "Multiple observers for different aspects of agent behavior",
                    "Flexible response to performance changes",
                    "Comprehensive analytics and reporting capabilities"
                ],
                "implementation": "BehaviorController monitoring agent performance and adapting traits"
            },

            "template_method_pattern": {
                "description": "Common agent behavior framework with customizable specializations",
                "benefits": [
                    "Consistent agent interface across different domains",
                    "Shared common functionality with domain-specific extensions",
                    "Easier maintenance and updates to common behavior",
                    "Standardized error handling and performance tracking"
                ],
                "implementation": "CustomAgentBase with abstract specialized_operation method"
            },

            "factory_pattern": {
                "description": "PersonalityTemplate for creating consistent behavioral profiles",
                "benefits": [
                    "Standardized personality creation across agents",
                    "Consistent behavioral profiles for similar roles",
                    "Easy replication of successful agent configurations",
                    "Simplified agent initialization and setup"
                ],
                "implementation": "PersonalityTemplate static methods for profile creation"
            }
        }

    def _define_use_cases(self) -> Dict[str, Dict[str, Any]]:
        """Define real-world use cases for the framework"""

        return {
            "financial_services": {
                "description": "Automated financial advisory and portfolio management",
                "scenarios": [
                    "Robo-advisor platforms with personalized investment strategies",
                    "Risk assessment systems for loan and credit decisions",
                    "Market analysis tools for institutional trading",
                    "Financial planning applications for retirement and goals"
                ],
                "agent_teams": [
                    "Conservative + Aggressive financial agents for balanced perspective",
                    "Risk assessment + Market prediction for comprehensive analysis",
                    "Portfolio analysis + Legal review for compliance checking"
                ]
            },

            "healthcare_systems": {
                "description": "Medical information systems and patient guidance platforms",
                "scenarios": [
                    "Symptom checker applications with emergency triage",
                    "Medication interaction and adherence monitoring systems",
                    "Health education platforms with personalized content",
                    "Clinical decision support tools for healthcare providers"
                ],
                "agent_teams": [
                    "Symptom analysis + Emergency triage for comprehensive assessment",
                    "Health education + Medication consultation for patient care",
                    "Multiple specialty agents for complex medical scenarios"
                ]
            },

            "legal_technology": {
                "description": "Legal document analysis and compliance management",
                "scenarios": [
                    "Contract review and risk assessment platforms",
                    "Legal research and case law analysis tools",
                    "Compliance monitoring and regulatory guidance systems",
                    "Document drafting and template generation services"
                ],
                "agent_teams": [
                    "Contract analysis + Risk assessment for comprehensive review",
                    "Legal research + Compliance checking for regulatory guidance",
                    "Multiple legal specialty agents for complex document analysis"
                ]
            },

            "enterprise_consulting": {
                "description": "Multi-domain consulting and advisory services",
                "scenarios": [
                    "Business strategy consulting with financial and legal analysis",
                    "Risk management consulting across multiple domains",
                    "Compliance consulting with regulatory and legal expertise",
                    "Decision support systems for complex business problems"
                ],
                "agent_teams": [
                    "Financial + Legal + Domain expert teams for comprehensive consulting",
                    "Conservative + Analytical agents for risk-aware recommendations",
                    "Hierarchical teams with lead consultant and specialist advisors"
                ]
            }
        }

    def generate_summary_report(self) -> str:
        """Generate comprehensive framework summary report"""

        report = []
        report.append("🎯 TUTORIAL 14 FRAMEWORK SUMMARY")
        report.append("=" * 35)
        report.append("")

        # Framework overview
        report.append("📚 FRAMEWORK COMPONENTS")
        report.append("-" * 25)

        for part_name, part_info in self.framework_components.items():
            report.append(f"\n{part_name.upper().replace('_', ' ')}")
            report.append(f"Description: {part_info['description']}")

            if 'key_classes' in part_info:
                report.append("Key Classes:")
                for class_name, class_info in part_info['key_classes'].items():
                    report.append(f"  • {class_name}: {class_info['purpose']}")

            if 'implemented_agents' in part_info:
                report.append("Implemented Agents:")
                for agent_name, agent_info in part_info['implemented_agents'].items():
                    report.append(f"  • {agent_name}: {agent_info['domain']}")

        # Learning outcomes
        report.append("\n\n🎓 LEARNING OUTCOMES")
        report.append("-" * 20)
        for outcome in self.learning_outcomes:
            report.append(outcome)

        # Architectural insights
        report.append("\n\n🏛️ ARCHITECTURAL INSIGHTS")
        report.append("-" * 27)
        for category, insights in self.architectural_insights.items():
            report.append(f"\n{category.replace('_', ' ').title()}:")
            for insight in insights[:3]:  # Limit to top 3 insights per category
                report.append(f"  • {insight}")

        # Design patterns
        report.append("\n\n🎨 KEY DESIGN PATTERNS")
        report.append("-" * 23)
        for pattern_name, pattern_info in self.design_patterns.items():
            report.append(f"\n{pattern_name.replace('_', ' ').title()}:")
            report.append(f"  {pattern_info['description']}")
            report.append(f"  Implementation: {pattern_info['implementation']}")

        # Use cases
        report.append("\n\n💼 REAL-WORLD USE CASES")
        report.append("-" * 25)
        for use_case, case_info in self.use_cases.items():
            report.append(f"\n{use_case.replace('_', ' ').title()}:")
            report.append(f"  {case_info['description']}")
            report.append(f"  Example scenarios: {len(case_info['scenarios'])} identified")

        return "\n".join(report)

# =============================================================================
# GENERATE FRAMEWORK SUMMARY
# =============================================================================

print("📋 Generating Framework Summary...")
print()

# Create summary object
framework_summary = FrameworkSummary()

# Generate comprehensive framework report
summary_report = framework_summary.generate_summary_report()
print(summary_report)

print("\n\n" + "="*60)
print()

# Additional detailed insights
print("🔍 DETAILED ARCHITECTURAL INSIGHTS")
print("=" * 37)
print()

print("💡 CAPABILITY SYSTEM INNOVATIONS:")
for insight in framework_summary.architectural_insights["capability_architecture"]:
    print(f"  • {insight}")

print("\n💡 TRAIT SYSTEM INNOVATIONS:")
for insight in framework_summary.architectural_insights["trait_system_design"]:
    print(f"  • {insight}")

print("\n💡 TEAM COORDINATION INNOVATIONS:")
for insight in framework_summary.architectural_insights["team_coordination_architecture"]:
    print(f"  • {insight}")

print("\n\n" + "="*60)
print()

print("🎯 FRAMEWORK SUCCESS METRICS")

print("=" * 30)

print()

print("✅ COMPONENT COVERAGE:")

print("  • 4 Core foundational classes (100% complete)")

print("  • 3 Specialized domain agents (Finance, Healthcare, Legal)")

print("  • 5 Behavioral control systems (Templates, Controllers, Composers)")

print("  • 3 Coordination patterns (Parallel, Sequential, Hierarchical)")

print("  • 3 Composition strategies (Optimal, Diverse, Specialist)")

print()

print("✅ FEATURE COMPLETENESS:")


print("  • Performance tracking and optimization ✓")

print("  • Context-aware behavioral adaptation ✓")

print("  • Intelligent team formation and coordination ✓")

print("  • Professional domain expertise with safety protocols ✓")

print("  • Comprehensive error handling and recovery ✓")

print("  • Production-ready patterns and scalability considerations ✓")

print()

print("✅ REAL-WORLD APPLICABILITY:")

print("  • Financial services and robo-advisory platforms ✓")

print("  • Healthcare information and decision support systems ✓")

print("  • Legal technology and document analysis platforms ✓")

print("  • Enterprise consulting and multi-domain advisory services ✓")

print("🎊 FRAMEWORK SUMMARY COMPLETE!")

print("Ready to implement sophisticated domain-specific agent systems")

print("with intelligent team coordination and dynamic behavior adaptation.")


Tutorial 14: Framework Summary & Architectural Insights
Complete framework overview and design insights...

📋 Generating Framework Summary...

🎯 TUTORIAL 14 FRAMEWORK SUMMARY

📚 FRAMEWORK COMPONENTS
-------------------------

PART 1 FOUNDATION
Description: Core agent specialization framework and building blocks
Key Classes:
  • AgentCapability: Executable skills with performance tracking and resource management
  • AgentTrait: Behavioral modifiers affecting decision-making and communication
  • DomainKnowledge: Expert knowledge bases with structured information storage
  • CustomAgentBase: Abstract foundation providing common agent functionality

PART 2 SPECIALIZATION
Description: Domain-specific agent implementations with professional expertise
Implemented Agents:
  • FinancialAnalystAgent: Financial analysis and advisory services
  • HealthcareAdvisorAgent: Healthcare guidance and medical information
  • LegalAdvisorAgent: Legal research and document analysis

PART 3 COMPOSITION
Desc

---

🔒 **INTELLECTUAL PROPERTY & LICENSE NOTICE**

This tutorial and its contents — including code, architecture, narrative examples, and educational structure — are the intellectual property of **Shalini Ananda, PhD** and part of the **Neuron Framework** under a **Modified MIT License with Attribution**.

- Commercial use, redistribution, or derivative works **must** include clear and visible attribution to the original author.
- Use in products, consulting engagements, or educational materials **must reference this repository and author name.**
- Removal of author credit or misrepresentation of origin constitutes **a violation of the license and may trigger legal action.**
- You may **not white-label, obfuscate, or rebrand** this work without explicit, written permission.

Use of this tutorial in Colab or any other platform implies agreement with these terms.

📘 **License**: [LICENSE.md](../LICENSE.md)  
📌 **Notice**: [NOTICE.md](../NOTICE.md)  
🧠 **Author**: [Shalini Ananda, PhD](https://github.com/ShaliniAnandaPhD)

---
